# Lab Exercise: High-Frequency Arbitrage in Fragmented Markets

**Deadline:** 9th of December 23:59 CET

**Submission:** Email to francisco.merlos@six-group.com with title: "Arbitrage study in BME | Your name"

In [ ]:
# Import necessary libraries for data processing and analysis
import pandas as pd
import numpy as np
import os
from pathlib import Path
from glob import glob
import warnings
warnings.filterwarnings('ignore')

# Import matplotlib for visualization
import matplotlib.pyplot as plt

# Set pandas display options for better readability
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)


## 1. Context: The Fragmented Market

In modern European equity markets, liquidity is **fragmented**. The same stock (ISIN) trades simultaneously on the primary exchange (BME) and various Multilateral Trading Facilities (MTFs) like CBOE, Turquoise, and Aquis.

Due to this fragmentation, temporary price discrepancies occur. A stock might be offered for sale at €10.00 on Turquoise while a buyer is bidding €10.01 on BME. A High-Frequency Trader (HFT) can profit from this by buying low and selling high instantaneously.

However, these opportunities are fleeting. The "theoretical" profit you see in a snapshot might disappear by the time your order reaches the exchange due to **latency**.

### The Mission

You have been hired as a Quantitative Researcher at a proprietary trading firm. Your boss has given you a dataset of high-resolution market data and asked you to answer three critical questions:

1. **Do arbitrage opportunities still exist in Spanish equities?**
2. **What is the maximum theoretical profit** (assuming 0 latency)?
3. **The "Latency Decay" Curve:** How quickly does this profit vanish as our trading system gets slower (from 0µs to 100ms)?

## 2. Data Specifications

You are provided with a `DATA_BIG/` folder containing subfolders for specific trading dates. Inside, you will find three types of compressed CSV files for various instruments.

**Note:** You can also find a `DATA_SMALL` folder that you can use to test quickly without needing to run the simulation over all the data.

### File Naming Convention

The naming pattern for all three file types (QTE, STS, TRD) is:

```
<type>_<session>_<isin>_<ticker>_<mic>_<part>.csv.gz
```

| Field | Description |
|-------|-------------|
| **type** | QTE, TRD, or STS |
| **session** | Trading date (YYYY-MM-DD) |
| **isin** | Cross-venue **ISIN** (International Securities Identification Number) |
| **ticker** | Venue-specific trading symbol (distinguishes multiple books for the same ISIN on the same MIC) |
| **mic** | Market Identifier Code (MIC, e.g., XMAD) |
| **part** | Integer part number. Assume it is always 1 for simplicity. |

### Order Book Identity and Join Key

A single **order book identity** is defined by the tuple:

```
(session, isin, mic, ticker)
```

This identity is the **key used to join** corresponding QTE, TRD, and STS data belonging to the same book.

### File Types

1. **QTE (Quotes/Snapshots):** Represents the state of the order book (up to 10 levels deep).
   - `epoch`: Timestamp in microseconds (UTC).
   - `px_bid_0`, `px_ask_0`: Best Bid and Best Ask prices.
   - `qty_bid_0`, `qty_ask_0`: Available volume at the best price.
   - *Note: Columns exist for levels 0-9.*

2. **STS (Trading Status):** Updates on the market phase (e.g., Open, Auction, Closed).
   - `epoch`: Timestamp.
   - `market_trading_status`: An integer code representing the state.

3. **TRD (Trades):** Represents the transactions. Not needed for this exercise.

### CRITICAL: Vendor Data Definitions

Real-world financial data is rarely clean. The data vendor has provided the following specifications. **Ignoring these will result in massive errors in your P&L calculation.**

#### A. "Magic Numbers" (Special Price Codes)

The vendor uses specific high-value constants to indicate non-tradable states (e.g., Market Orders during auctions). **These are NOT real prices.** If you treat 999,999 as a valid bid, your algorithm will assume you can sell for a million euros.

| Value | Meaning | Action Required |
|-------|---------|----------------|
| 666666.666 | Unquoted/Unknown | **Discard** |
| 999999.999 | Market Order (At Best) | **Discard** |
| 999999.989 | At Open Order | **Discard** |
| 999999.988 | At Close Order | **Discard** |
| 999999.979 | Pegged Order | **Discard** |
| 999999.123 | Unquoted/Unknown | **Discard** |

#### B. Market Status Codes

You can only trade when the market is in **Continuous Trading**. If you trade during an Auction, a Halt, or Pre-Open, your order will not execute immediately. A snapshot is only valid/addressable if the STS for that venue is one of these codes:

| Venue | Continuous Trading Code |
|-------|------------------------|
| AQUIS | 5308427 |
| BME | 5832713, 5832756 |
| CBOE | 12255233 |
| TURQUOISE | 7608181 |

## 3. Implementation Guide

You are encouraged to use AI tools (ChatGPT, Claude, etc.) to generate the Python/Pandas code. However, **you** are responsible for the logic and the financial validity of the results.

### Step 1: Data Ingestion & Cleaning

- Write a function to load the QTE and STS files for a given ISIN.
- **Task:** Ensure you are using only valid prices
- **Task:** Ensure you are only looking at addressable orderbooks

In [ ]:
# ============================================================================
# STEP 1: DATA INGESTION & CLEANING
# ============================================================================
# This step discovers all ISINs in DATA_BIG and loads QTE (quotes) and STS 
# (trading status) files for each ISIN across all venues.
# ============================================================================

# Define the base data directory and trading session
DATA_DIR = 'DATA_BIG'
SESSION = '2025-11-07'

# Define venues (Market Identifier Codes) we need to process
VENUES = ['BME', 'CBOE', 'TURQUOISE', 'AQUIS']

# Define "magic numbers" that represent invalid/non-tradable prices
# These are special codes used by the vendor to indicate non-executable states
INVALID_PRICES = [
    666666.666,  # Unquoted/Unknown
    999999.999,  # Market Order (At Best)
    999999.989,  # At Open Order
    999999.988,  # At Close Order
    999999.979,  # Pegged Order
    999999.123   # Unquoted/Unknown
]

# Define valid market trading status codes for continuous trading per venue
# Only orderbooks in continuous trading are "addressable" (can be traded)
VALID_STATUS_CODES = {
    'AQUIS': [5308427],
    'BME': [5832713, 5832756],
    'CBOE': [12255233],
    'TURQUOISE': [7608181]
}

def discover_all_isins(data_dir=DATA_DIR, session=SESSION, venues=VENUES):
    """
    Discovers all unique ISINs across all venues by scanning QTE files.
    
    In fragmented markets, the same ISIN trades on multiple venues. This function
    scans all venue folders to build a comprehensive list of all available instruments.
    
    Returns:
        set: Unique ISINs found across all venues
    """
    isins = set()
    
    # Scan each venue folder for QTE files
    for venue in venues:
        venue_folder = f"{data_dir}/{venue}_{session}"
        
        if not os.path.exists(venue_folder):
            print(f"Warning: Venue folder {venue_folder} not found. Skipping.")
            continue
        
        # Find all QTE files in this venue
        # File pattern: QTE_<session>_<isin>_<ticker>_<mic>_<part>.csv.gz
        qte_files = glob(f"{venue_folder}/QTE_*.csv.gz")
        
        # Extract ISIN from each filename (3rd component after splitting by '_')
        for file_path in qte_files:
            filename = os.path.basename(file_path)
            parts = filename.split('_')
            if len(parts) >= 3:
                isin = parts[2]  # ISIN is the 3rd component
                isins.add(isin)
    
    print(f"Discovered {len(isins)} unique ISINs across all venues")
    return sorted(isins)

def load_qte_sts_for_isin(isin, data_dir=DATA_DIR, session=SESSION, venues=VENUES):
    """
    Loads all QTE (quotes) and STS (trading status) files for a given ISIN across all venues.
    
    Market Microstructure Context:
    - QTE files contain orderbook snapshots showing bid/ask prices and quantities
    - STS files contain market phase information (Open, Continuous Trading, Auction, etc.)
    - We need both to determine: 1) What prices are available, 2) Whether we can actually trade
    
    Args:
        isin: The ISIN identifier (e.g., 'ES0113900J37')
        data_dir: Base directory containing venue folders
        session: Trading date (YYYY-MM-DD)
        venues: List of venue codes to process
    
    Returns:
        tuple: (qte_dataframes_dict, sts_dataframes_dict)
            - qte_dataframes_dict: {venue: DataFrame} mapping
            - sts_dataframes_dict: {venue: DataFrame} mapping
    """
    qte_dfs = {}
    sts_dfs = {}
    
    # Process each venue
    for venue in venues:
        venue_folder = f"{data_dir}/{venue}_{session}"
        
        if not os.path.exists(venue_folder):
            continue
        
        # Find QTE and STS files for this ISIN in this venue
        # Note: There may be multiple tickers for the same ISIN on the same venue
        qte_pattern = f"{venue_folder}/QTE_{session}_{isin}_*_*.csv.gz"
        sts_pattern = f"{venue_folder}/STS_{session}_{isin}_*_*.csv.gz"
        
        qte_files = glob(qte_pattern)
        sts_files = glob(sts_pattern)
        
        # Load all QTE files for this venue (may have multiple orderbooks)
        if qte_files:
            # Load and concatenate all QTE files for this ISIN+venue combination
            qte_list = []
            for qte_file in qte_files:
                try:
                    df_qte = pd.read_csv(qte_file, sep=';', compression='gzip')
                    # Add metadata columns to identify the orderbook
                    filename = os.path.basename(qte_file)
                    parts = filename.split('_')
                    if len(parts) >= 5:
                        df_qte['ticker'] = parts[3]  # Extract ticker from filename
                        df_qte['mic'] = parts[4]     # Extract MIC from filename
                        df_qte['venue'] = venue      # Add venue identifier
                    qte_list.append(df_qte)
                except Exception as e:
                    print(f"Error loading {qte_file}: {e}")
                    continue
            
            if qte_list:
                # Combine all QTE files for this venue
                qte_combined = pd.concat(qte_list, ignore_index=True)
                qte_dfs[venue] = qte_combined
                print(f"  {venue}: Loaded {len(qte_combined)} QTE records")
        
        # Load all STS files for this venue
        if sts_files:
            sts_list = []
            for sts_file in sts_files:
                try:
                    df_sts = pd.read_csv(sts_file, sep=';', compression='gzip')
                    filename = os.path.basename(sts_file)
                    parts = filename.split('_')
                    if len(parts) >= 5:
                        df_sts['ticker'] = parts[3]
                        df_sts['mic'] = parts[4]
                        df_sts['venue'] = venue
                    sts_list.append(df_sts)
                except Exception as e:
                    print(f"Error loading {sts_file}: {e}")
                    continue
            
            if sts_list:
                sts_combined = pd.concat(sts_list, ignore_index=True)
                sts_dfs[venue] = sts_combined
                print(f"  {venue}: Loaded {len(sts_combined)} STS records")
    
    return qte_dfs, sts_dfs

# ============================================================================
# EXECUTE: Discover all ISINs and load QTE/STS files
# ============================================================================

print("=" * 70)
print("STEP 1: DISCOVERING ISINs AND LOADING QTE/STS FILES")
print("=" * 70)

# Discover all unique ISINs in the dataset
all_isins = discover_all_isins()
print(f"\nTotal unique ISINs found: {len(all_isins)}")
print(f"First 10 ISINs: {all_isins[:10]}\n")

# Load QTE and STS files for each ISIN
# Note: We'll process all ISINs, but for large datasets, you might want to 
# process in batches or use lazy loading strategies
print("\nLoading QTE and STS files for all ISINs...")
print("-" * 70)

# Store all loaded data in a dictionary structure
# Structure: {isin: {'qte': {venue: df}, 'sts': {venue: df}}}
all_data = {}

for idx, isin in enumerate(all_isins, 1):
    print(f"\n[{idx}/{len(all_isins)}] Processing ISIN: {isin}")
    qte_dfs, sts_dfs = load_qte_sts_for_isin(isin)
    
    if qte_dfs or sts_dfs:
        all_data[isin] = {
            'qte': qte_dfs,
            'sts': sts_dfs
        }
    else:
        print(f"  Warning: No QTE/STS files found for {isin}")

print("\n" + "=" * 70)
print(f"STEP 1 COMPLETE: Loaded data for {len(all_data)} ISINs")
print("=" * 70)


In [ ]:
# ============================================================================
# DATA CLEANING & PREPARATION: Unified Function Following Reference Pattern
# ============================================================================
# This unified function combines cleaning and timestamp preparation following
# the structure from 'Working with microsecond timeseries.py' (clean_hft_data_preserve_all).
# 
# The function:
# 1. Sorts by ['epoch', 'sequence'] FIRST (before any operations)
# 2. Applies nanosecond trick to create unique timestamps
# 3. Sets timestamp index
# 4. Merges STS status and filters invalid prices/status codes
# ============================================================================

def clean_and_prepare_qte_data(qte_df, sts_df=None, venue=None, 
                                invalid_prices=INVALID_PRICES, 
                                valid_status_codes=VALID_STATUS_CODES):
    """
    Unified function that cleans and prepares QTE data following the reference pattern.
    
    This function follows the structure from 'clean_hft_data_preserve_all' in the reference:
    1. Sorts strictly by epoch and sequence (if available)
    2. Applies nanosecond trick to handle duplicate epochs
    3. Sets unique timestamp index
    4. Merges STS trading status (if available)
    5. Filters invalid prices and non-continuous trading periods
    
    Market Microstructure Context:
    - Multiple orderbook snapshots can occur at the same microsecond
    - Sequence column preserves the order of events within the same microsecond
    - We can only trade during Continuous Trading (not during auctions/halts)
    - Magic numbers represent non-executable orders (market orders, pegged orders, etc.)
    - These must be filtered out to avoid false arbitrage signals
    
    Args:
        qte_df: DataFrame with QTE (quote) data (must have 'epoch' column)
        sts_df: Optional DataFrame with STS (trading status) data
        venue: Optional venue identifier (BME, CBOE, etc.) for status code filtering
        invalid_prices: List of magic numbers to filter out
        valid_status_codes: Dict mapping venue to valid status codes
    
    Returns:
        DataFrame with unique timestamp index, cleaned and filtered
    """
    if qte_df.empty:
        return qte_df
    
    # Make a copy to avoid modifying original
    df = qte_df.copy()
    
    # ========================================================================
    # STEP 1: Sort strictly by epoch and sequence (if available)
    # ========================================================================
    # This is CRITICAL: We must sort by ['epoch', 'sequence'] BEFORE any
    # time-based operations. The sequence column preserves the order of events
    # within the same microsecond, which is essential for accurate time ordering.
    # 
    # Market Microstructure: When multiple orderbook updates happen at the
    # same microsecond, the sequence number indicates the order in which they
    # occurred. This is crucial for maintaining chronological accuracy.
    sort_cols = ['epoch']
    if 'sequence' in df.columns:
        sort_cols.append('sequence')
        # Note: QTE files include sequence column to preserve event order
    else:
        # If sequence is missing, document why (shouldn't happen for QTE files)
        print("    Warning: 'sequence' column not found in QTE data. Using epoch only for sorting.")
    
    df = df.sort_values(by=sort_cols, ascending=[True] * len(sort_cols))
    
    # ========================================================================
    # STEP 2: Apply the "Nanosecond Trick" to create unique timestamps
    # ========================================================================
    # Convert epoch to datetime (microseconds)
    temp_ts = pd.to_datetime(df['epoch'], unit='us')
    
    # The "Nanosecond Trick" (from reference: clean_hft_data_preserve_all)
    # groupby().cumcount() numbers items in a group: 0, 1, 2, 3...
    # We group by 'epoch' to find snapshots happening at the same time.
    # We treat that count as nanoseconds to create unique timestamps.
    offset_ns = df.groupby('epoch').cumcount()
    
    # Safety check: ensure we don't exceed 1000 snapshots per microsecond
    # (nanoseconds go from 0-999, so 1000 would overflow to next microsecond)
    max_offset = offset_ns.max() if len(offset_ns) > 0 else 0
    if max_offset >= 1000:
        raise Exception(f"Too many snapshots at the same microsecond. Max offset: {max_offset}")
    
    # Create the final High-Resolution Timestamp
    # Base Time (microseconds) + Offset (nanoseconds)
    df['ts'] = temp_ts + pd.to_timedelta(offset_ns, unit='ns')
    
    # ========================================================================
    # STEP 3: Set timestamp as index
    # ========================================================================
    # This creates a unique timestamp index that can be used for merge_asof,
    # pivot operations, and time-based lookups
    df.set_index('ts', inplace=True)
    
    # ========================================================================
    # STEP 4: Merge STS trading status (if available)
    # ========================================================================
    # Process STS data if available to filter by market trading status
    if sts_df is not None and not sts_df.empty:
        # Prepare STS data for merging
        sts_clean = sts_df.copy()
        sts_clean['ts'] = pd.to_datetime(sts_clean['epoch'], unit='us')
        
        # Sort STS by timestamp for merge_asof
        sts_clean = sts_clean.sort_values('ts').reset_index(drop=True)
        
        # Get valid status codes for this venue
        venue_valid_codes = valid_status_codes.get(venue, []) if venue else []
        
        # Merge STS status with QTE using merge_asof (backward direction)
        # This assigns the most recent market status to each quote
        # Direction='backward' ensures we only use status that was known at quote time
        # (avoids look-ahead bias)
        df = pd.merge_asof(
            df.reset_index(),  # Reset index temporarily for merge_asof
            sts_clean[['ts', 'market_trading_status']],
            on='ts',
            direction='backward'
        )
        
        # Filter: Keep only rows where market status is in continuous trading
        if venue_valid_codes:
            df = df[df['market_trading_status'].isin(venue_valid_codes)].copy()
        
        # Drop the market_trading_status column (no longer needed)
        df = df.drop(columns=['market_trading_status'], errors='ignore')
        
        # Set timestamp back as index
        df.set_index('ts', inplace=True)
    elif venue:
        # If no STS data but venue is specified, warn but continue
        print(f"    Warning: No STS data for {venue}, cannot filter by trading status")
    
    # ========================================================================
    # STEP 5: Filter out invalid prices (magic numbers)
    # ========================================================================
    # Check both bid and ask prices at level 0 (best bid/ask)
    price_columns = ['px_bid_0', 'px_ask_0']
    
    # Create a mask: True for rows with valid prices
    valid_price_mask = pd.Series(True, index=df.index)
    
    for col in price_columns:
        if col in df.columns:
            # Check if price is in the invalid prices list
            col_mask = ~df[col].isin(invalid_prices)
            valid_price_mask = valid_price_mask & col_mask
    
    # Apply the filter
    df = df[valid_price_mask].copy()
    
    return df

# ============================================================================
# EXECUTE: Clean all loaded QTE data
# ============================================================================

print("\n" + "=" * 70)
print("CLEANING & PREPARING QTE DATA: Filtering Invalid Prices, Non-Continuous Trading, and Creating Unique Timestamps")
print("=" * 70)

# Clean QTE data for each ISIN
cleaned_data = {}

for isin, data in all_data.items():
    print(f"\nCleaning and preparing data for ISIN: {isin}")
    qte_dfs = data.get('qte', {})
    sts_dfs = data.get('sts', {})
    
    cleaned_qte = {}
    
    for venue in qte_dfs.keys():
        qte_df = qte_dfs[venue]
        sts_df = sts_dfs.get(venue, pd.DataFrame())
        
        # Get original count
        original_count = len(qte_df)
        
        # Clean and prepare the data (unified function)
        qte_cleaned = clean_and_prepare_qte_data(qte_df, sts_df, venue)
        
        # Get cleaned count
        cleaned_count = len(qte_cleaned)
        removed_count = original_count - cleaned_count
        
        if original_count > 0:
            removal_pct = (removed_count / original_count) * 100
            print(f"  {venue}: {original_count:,} -> {cleaned_count:,} records "
                  f"({removed_count:,} removed, {removal_pct:.2f}%)")
        
        if cleaned_count > 0:
            cleaned_qte[venue] = qte_cleaned
    
    if cleaned_qte:
        cleaned_data[isin] = {
            'qte': cleaned_qte,
            'sts': sts_dfs  # STS data unchanged
        }

print("\n" + "=" * 70)
print(f"CLEANING & PREPARATION COMPLETE: Processed data for {len(cleaned_data)} ISINs")
print("=" * 70)

# Update all_data with cleaned and prepared version
all_data = cleaned_data


In [ ]:
# ============================================================================
# NOTE: Timestamp Preparation is Now Integrated
# ============================================================================
# The timestamp preparation (nanosecond trick) is now integrated into the
# unified clean_and_prepare_qte_data() function in the previous cell.
# This ensures proper sorting by ['epoch', 'sequence'] before any operations.
# 
# The data in all_data is already cleaned and prepared with unique timestamps.
# ============================================================================

# Verification: Check that timestamps are unique
print("\n" + "=" * 70)
print("VERIFICATION: Checking Timestamp Uniqueness")
print("=" * 70)

unique_check_passed = True
for isin, data in all_data.items():
    qte_dfs = data.get('qte', {})
    for venue, qte_df in qte_dfs.items():
        if not qte_df.empty:
            is_unique = qte_df.index.is_unique
            if not is_unique:
                print(f"  WARNING: {isin} on {venue} has non-unique timestamps!")
                unique_check_passed = False

if unique_check_passed:
    print("  ✓ All timestamps are unique across all ISINs and venues")
else:
    print("  ✗ Some timestamps are not unique - please check the data")

# Note: The old prepare_qte_timestamps function has been removed as it's now
# integrated into clean_and_prepare_qte_data() following the reference pattern.

# ============================================================================
# DEMONSTRATION: Nanosecond Trick for Duplicate Epochs
# ============================================================================
# Similar to the reference notebook, we demonstrate how the nanosecond trick
# handles multiple snapshots at the same microsecond
# ============================================================================

print("\n" + "=" * 70)
print("DEMONSTRATION: Handling Duplicate Epochs with Nanosecond Trick")
print("=" * 70)

# Find an example with duplicate epochs (one that needed the nanosecond trick)
example_found = False
example_isin = None
example_venue = None
example_df = None

for isin, data in all_data.items():
    qte_dfs = data.get('qte', {})
    for venue, qte_df in qte_dfs.items():
        if not qte_df.empty:
            # Check for duplicate epochs (before preparation, but we can check the index)
            # Look for timestamps with nanosecond offsets (indicating duplicates were resolved)
            bursts = qte_df[qte_df.index.nanosecond > 0]
            if not bursts.empty:
                example_isin = isin
                example_venue = venue
                example_df = qte_df
                example_found = True
                break
    if example_found:
        break

if example_found and example_df is not None:
    print(f"\n📊 Example: {example_isin} on {example_venue}")
    print(f"   Total snapshots: {len(example_df):,}")
    
    # Check for duplicate epochs (like in reference notebook lines 1-3)
    # We need to check the original epoch column to see duplicates
    duplicates = example_df.duplicated(subset='epoch', keep=False).sum()
    print(f"   Found {duplicates} snapshots sharing the same microsecond (duplicate epochs)")
    
    # Find a specific burst example
    bursts = example_df[example_df.index.nanosecond > 0]
    if not bursts.empty:
        print(f"   Found {len(bursts)} snapshots with nanosecond offsets (collisions resolved)")
        
        # Get the first burst timestamp
        burst_ts = bursts.index[0]
        base_ts = burst_ts.replace(nanosecond=0)
        
        # Get all snapshots at this base timestamp (the "burst")
        burst_snapshots = example_df.loc[base_ts : base_ts + pd.Timedelta(nanoseconds=999)]
        
        print(f"\n💥 Example of a resolved collision (Look at the timestamps!):")
        print(f"   Base timestamp: {base_ts}")
        print(f"   Number of snapshots at this microsecond: {len(burst_snapshots)}")
        print(f"\n   All snapshots at this microsecond:")
        
        # Show relevant columns including sequence if available
        display_cols = ['epoch', 'px_bid_0', 'px_ask_0', 'qty_bid_0', 'qty_ask_0']
        if 'sequence' in burst_snapshots.columns:
            display_cols.insert(1, 'sequence')
        
        print(burst_snapshots[display_cols].to_string())
        
        print(f"\n   Note: Each snapshot has a unique timestamp index with nanosecond offsets")
        print(f"   This preserves the order (by sequence if available) while ensuring uniqueness")
    else:
        print("   No duplicate epochs found in this example")
else:
    print("\n⚠️  No examples with duplicate epochs found to demonstrate")




In [ ]:
# ============================================================================
# DESCRIPTIVE SUMMARY: Overview of Loaded Data
# ============================================================================
# This section creates a summary DataFrame showing key statistics about the
# downloaded data: venues, ISINs, record counts, etc.
# ============================================================================

print("\n" + "=" * 70)
print("DESCRIPTIVE SUMMARY OF LOADED DATA")
print("=" * 70)

# Initialize summary statistics
summary_data = []

# Analyze each ISIN
for isin, data in all_data.items():
    qte_dfs = data.get('qte', {})
    sts_dfs = data.get('sts', {})
    
    # Count records per venue for this ISIN
    for venue in VENUES:
        qte_count = len(qte_dfs.get(venue, pd.DataFrame()))
        sts_count = len(sts_dfs.get(venue, pd.DataFrame()))
        
        if qte_count > 0 or sts_count > 0:
            summary_data.append({
                'ISIN': isin,
                'Venue': venue,
                'QTE_Records': qte_count,
                'STS_Records': sts_count,
                'Total_Records': qte_count + sts_count
            })

# Create summary DataFrame
df_summary = pd.DataFrame(summary_data)

if len(df_summary) > 0:
    # Overall statistics
    print(f"\n📊 OVERALL STATISTICS:")
    print(f"   • Total unique ISINs: {df_summary['ISIN'].nunique()}")
    print(f"   • Total venue-ISIN combinations: {len(df_summary)}")
    print(f"   • Total QTE records: {df_summary['QTE_Records'].sum():,}")
    print(f"   • Total STS records: {df_summary['STS_Records'].sum():,}")
    print(f"   • Total records: {df_summary['Total_Records'].sum():,}")
    
    # Statistics by venue
    print(f"\n🏢 STATISTICS BY VENUE:")
    venue_stats = df_summary.groupby('Venue').agg({
        'ISIN': 'nunique',
        'QTE_Records': 'sum',
        'STS_Records': 'sum',
        'Total_Records': 'sum'
    }).rename(columns={'ISIN': 'Unique_ISINs'})
    venue_stats = venue_stats.sort_values('Total_Records', ascending=False)
    print(venue_stats.to_string())
    
    # Statistics by ISIN (top 10)
    print(f"\n📈 TOP 10 ISINs BY TOTAL RECORDS:")
    isin_stats = df_summary.groupby('ISIN').agg({
        'Venue': 'count',
        'QTE_Records': 'sum',
        'STS_Records': 'sum',
        'Total_Records': 'sum'
    }).rename(columns={'Venue': 'Num_Venues'})
    isin_stats = isin_stats.sort_values('Total_Records', ascending=False)
    print(isin_stats.head(10).to_string())
    
    # Venue coverage per ISIN
    print(f"\n🔗 VENUE COVERAGE:")
    venue_coverage = df_summary.groupby('ISIN')['Venue'].count()
    coverage_stats = venue_coverage.value_counts().sort_index()
    print("Number of venues per ISIN:")
    for num_venues, count in coverage_stats.items():
        print(f"   • {num_venues} venue(s): {count} ISIN(s)")
    
    # Statistics about duplicate epochs (nanosecond trick usage)
    print(f"\n⏱️  DUPLICATE EPOCHS STATISTICS (Nanosecond Trick Required):")
    duplicate_epoch_stats = []
    
    for isin, data in all_data.items():
        qte_dfs = data.get('qte', {})
        for venue, qte_df in qte_dfs.items():
            if not qte_df.empty:
                # Count snapshots with nanosecond offsets (indicating duplicate epochs)
                bursts = qte_df[qte_df.index.nanosecond > 0]
                num_bursts = len(bursts)
                
                if num_bursts > 0:
                    # Count unique epochs that had duplicates
                    burst_epochs = qte_df.loc[bursts.index, 'epoch'].unique()
                    num_duplicate_epochs = len(burst_epochs)
                    
                    # Find max number of snapshots at a single epoch
                    epoch_counts = qte_df.groupby('epoch').size()
                    max_snapshots_per_epoch = epoch_counts.max()
                    
                    duplicate_epoch_stats.append({
                        'ISIN': isin,
                        'Venue': venue,
                        'Snapshots_With_NS_Offset': num_bursts,
                        'Unique_Epochs_With_Duplicates': num_duplicate_epochs,
                        'Max_Snapshots_Per_Epoch': max_snapshots_per_epoch
                    })
    
    if duplicate_epoch_stats:
        df_dup_epochs = pd.DataFrame(duplicate_epoch_stats)
        total_dup_combinations = len(df_dup_epochs)
        total_snapshots_with_offset = df_dup_epochs['Snapshots_With_NS_Offset'].sum()
        total_dup_epochs = df_dup_epochs['Unique_Epochs_With_Duplicates'].sum()
        max_snapshots = df_dup_epochs['Max_Snapshots_Per_Epoch'].max()
        
        print(f"   • Venue-ISIN combinations with duplicate epochs: {total_dup_combinations}")
        print(f"   • Total snapshots requiring nanosecond offsets: {total_snapshots_with_offset:,}")
        print(f"   • Total unique epochs with duplicates: {total_dup_epochs:,}")
        print(f"   • Maximum snapshots at a single epoch: {max_snapshots}")
        print(f"\n   Top 5 by number of duplicate epochs:")
        top_5_dup = df_dup_epochs.nlargest(5, 'Unique_Epochs_With_Duplicates')
        print(top_5_dup[['ISIN', 'Venue', 'Unique_Epochs_With_Duplicates', 'Max_Snapshots_Per_Epoch']].to_string(index=False))
    else:
        print(f"   • No duplicate epochs found - all snapshots have unique microsecond timestamps")
    
    # Create a detailed summary DataFrame for display
    print(f"\n📋 DETAILED SUMMARY DATAFRAME (First 20 rows):")
    print(df_summary.head(20).to_string(index=False))
    
    # Store the summary DataFrame for later use
    print(f"\n✅ Summary DataFrame stored as 'df_summary' ({len(df_summary)} rows)")
    
else:
    print("⚠️  No data loaded. Please check Step 1 execution.")


# ============================================================================
# STATISTICS: Epochs Requiring Nanosecond Trick
# ============================================================================

print("\n" + "=" * 70)
print("STATISTICS: Epochs Requiring Nanosecond Trick")
print("=" * 70)

# Collect statistics about duplicate epochs
duplicate_stats = []

for isin, data in all_data.items():
    qte_dfs = data.get('qte', {})
    for venue, qte_df in qte_dfs.items():
        if not qte_df.empty:
            # Count snapshots with nanosecond offsets (indicating duplicate epochs were resolved)
            bursts = qte_df[qte_df.index.nanosecond > 0]
            num_bursts = len(bursts)
            
            if num_bursts > 0:
                # Count unique epochs that had duplicates
                burst_epochs = qte_df.loc[bursts.index, 'epoch'].unique()
                num_duplicate_epochs = len(burst_epochs)
                
                # Find max number of snapshots at a single epoch
                epoch_counts = qte_df.groupby('epoch').size()
                max_snapshots_per_epoch = epoch_counts.max()
                
                duplicate_stats.append({
                    'ISIN': isin,
                    'Venue': venue,
                    'Total_Snapshots': len(qte_df),
                    'Snapshots_With_NS_Offset': num_bursts,
                    'Unique_Epochs_With_Duplicates': num_duplicate_epochs,
                    'Max_Snapshots_Per_Epoch': max_snapshots_per_epoch
                })

if duplicate_stats:
    df_duplicate_stats = pd.DataFrame(duplicate_stats)
    
    print(f"\n📈 Summary of Epochs Requiring Nanosecond Trick:")
    print(f"   • Total venue-ISIN combinations with duplicate epochs: {len(df_duplicate_stats)}")
    print(f"   • Total snapshots requiring nanosecond offsets: {df_duplicate_stats['Snapshots_With_NS_Offset'].sum():,}")
    print(f"   • Total unique epochs with duplicates: {df_duplicate_stats['Unique_Epochs_With_Duplicates'].sum():,}")
    print(f"   • Maximum snapshots at a single epoch: {df_duplicate_stats['Max_Snapshots_Per_Epoch'].max()}")
    
    print(f"\n📋 Top 10 ISIN-Venue combinations by number of duplicate epochs:")
    top_duplicates = df_duplicate_stats.nlargest(10, 'Unique_Epochs_With_Duplicates')
    print(top_duplicates[['ISIN', 'Venue', 'Total_Snapshots', 'Unique_Epochs_With_Duplicates', 
                          'Max_Snapshots_Per_Epoch']].to_string(index=False))
    
    print(f"\n💡 Market Microstructure Context:")
    print(f"   Multiple orderbook snapshots at the same microsecond occur when:")
    print(f"   - Rapid orderbook updates happen faster than microsecond resolution")
    print(f"   - Market data feeds batch multiple updates together")
    print(f"   - High-frequency trading activity creates bursts of updates")
    print(f"   The nanosecond trick preserves all snapshots while maintaining chronological order")
else:
    print("\n✅ No duplicate epochs found - all snapshots have unique microsecond timestamps")


### Step 2: Create the "Consolidated Tape"

- To detect arbitrage, you need to compare prices across venues *at the exact same time*.
- **Task:** Create a single DataFrame where the index is the timestamp, and the columns represent the Best Bid and Best Ask for **every** venue (BME, XMAD, CBOE, etc.).

In [ ]:
# ============================================================================
# STEP 2: CREATE THE CONSOLIDATED TAPE
# ============================================================================
# The consolidated tape combines orderbook snapshots from all venues into
# a single time-indexed DataFrame, allowing us to compare prices across
# venues at the exact same timestamp.
#
# Market Microstructure Context:
# - Financial markets are asynchronous: updates happen at different times
#   on different venues
# - To detect arbitrage, we need to know what prices are available on ALL
#   venues at the SAME moment in time
# - We use "Last Traded Price" (LTP) logic: if a venue hasn't updated at
#   time T, we use the last known price from that venue
# - Forward fill (ffill) propagates the last known bid/ask forward until
#   a new update arrives
# ============================================================================

def create_consolidated_tape_for_isin(isin, cleaned_data_dict):
    """
    Creates a consolidated tape for a single ISIN across all venues.
    
    The consolidated tape is a DataFrame where:
    - Index: Timestamp (nanosecond precision)
    - Columns: Best Bid and Best Ask for each venue
      (e.g., 'BME_bid', 'BME_ask', 'CBOE_bid', 'CBOE_ask', etc.)
    
    Market Microstructure Context:
    - Each venue updates its orderbook independently
    - We need to see all venues' prices at the same timestamp to detect
      arbitrage opportunities
    - Forward fill ensures that if BME updates at t=100µs and CBOE updates
      at t=105µs, we can still compare prices at t=102µs using BME's last
      known price
    
    Args:
        isin: The ISIN identifier
        cleaned_data_dict: Dictionary with structure {isin: {'qte': {venue: df}}}
    
    Returns:
        DataFrame with timestamp index and columns for each venue's bid/ask
    """
    if isin not in cleaned_data_dict:
        return pd.DataFrame()
    
    qte_dfs = cleaned_data_dict[isin].get('qte', {})
    
    if not qte_dfs:
        return pd.DataFrame()
    
    # ========================================================================
    # STEP 1: Extract Best Bid and Ask from Each Venue
    # ========================================================================
    # For each venue, we extract px_bid_0 (best bid) and px_ask_0 (best ask)
    # along with their quantities. We also preserve the venue identifier
    # so we can distinguish between venues after concatenation.
    
    venue_data_list = []
    
    for venue, qte_df in qte_dfs.items():
        if qte_df.empty:
            continue
        
        # Extract only the columns we need: best bid/ask prices and quantities
        # Market Microstructure: We use level 0 (best bid/ask) because these
        # are the prices at which we can immediately execute trades
        venue_subset = qte_df[['px_bid_0', 'px_ask_0', 'qty_bid_0', 'qty_ask_0']].copy()
        
        # Add venue identifier as a column (needed for pivot operation)
        venue_subset['venue'] = venue
        
        # Rename columns to include venue prefix for clarity
        # This will help us identify which venue each price comes from
        venue_subset = venue_subset.rename(columns={
            'px_bid_0': 'bid',
            'px_ask_0': 'ask',
            'qty_bid_0': 'bid_qty',
            'qty_ask_0': 'ask_qty'
        })
        
        venue_data_list.append(venue_subset)
    
    if not venue_data_list:
        return pd.DataFrame()
    
    # ========================================================================
    # STEP 2: Combine All Venues into Single DataFrame
    # ========================================================================
    # Concatenate all venue data into one long DataFrame, preserving the
    # timestamp index from each venue's cleaned QTE data.
    # 
    # Market Microstructure: This creates a unified timeline where we can
    # see all orderbook updates from all venues in chronological order.
    
    tape_full = pd.concat(venue_data_list)
    
    # Sort by timestamp to ensure chronological order
    # This is critical: we need events in time order for forward fill to work
    tape_full.sort_index(inplace=True)
    
    # ========================================================================
    # STEP 3: Create Separate DataFrames for Bid and Ask
    # ========================================================================
    # We need to pivot bid and ask separately because pivot can only handle
    # one value column at a time. We'll create two pivoted DataFrames and
    # then combine them.
    
    # Pivot bids: creates columns like 'BME', 'CBOE', etc. with bid prices
    # Market Microstructure: Each column represents the best bid price
    # available on that venue at each timestamp
    tape_bid_pivot = tape_full.pivot(columns='venue', values='bid')
    
    # Pivot asks: creates columns like 'BME', 'CBOE', etc. with ask prices
    # Market Microstructure: Each column represents the best ask price
    # available on that venue at each timestamp
    tape_ask_pivot = tape_full.pivot(columns='venue', values='ask')
    
    # ========================================================================
    # STEP 4: Forward Fill to Propagate Last Known Prices
    # ========================================================================
    # Forward fill (ffill) is the key operation that makes the consolidated
    # tape work. It propagates the last known price forward until a new
    # update arrives.
    #
    # Market Microstructure: This implements the "Last Traded Price" (LTP)
    # logic. If BME updates at t=100µs and CBOE updates at t=105µs, then:
    # - At t=100µs: BME has new price, CBOE still has old price
    # - At t=101-104µs: Both venues have their last known prices
    # - At t=105µs: CBOE updates, BME still has price from t=100µs
    #
    # This allows us to compare prices across venues at ANY timestamp,
    # not just when all venues update simultaneously.
    
    tape_bid_ffill = tape_bid_pivot.ffill()
    tape_ask_ffill = tape_ask_pivot.ffill()
    
    # ========================================================================
    # STEP 5: Rename Columns to Include Bid/Ask Suffix
    # ========================================================================
    # Rename columns to make it clear which is bid and which is ask
    # This creates columns like 'BME_bid', 'BME_ask', 'CBOE_bid', etc.
    
    tape_bid_ffill.columns = [f'{venue}_bid' for venue in tape_bid_ffill.columns]
    tape_ask_ffill.columns = [f'{venue}_ask' for venue in tape_ask_ffill.columns]
    
    # ========================================================================
    # STEP 6: Combine Bid and Ask into Final Consolidated Tape
    # ========================================================================
    # Join the bid and ask DataFrames together to create the final consolidated
    # tape with all venue prices in one place.
    
    consolidated_tape = pd.concat([tape_bid_ffill, tape_ask_ffill], axis=1)
    
    # Sort columns alphabetically for consistency (optional, but makes output cleaner)
    consolidated_tape = consolidated_tape.sort_index(axis=1)
    
    return consolidated_tape

# ============================================================================
# EXECUTE: Create Consolidated Tape for All ISINs
# ============================================================================

print("\n" + "=" * 70)
print("STEP 2: CREATING CONSOLIDATED TAPE FOR ALL ISINs")
print("=" * 70)

# Store consolidated tapes for all ISINs
consolidated_tapes = {}

# Process each ISIN
for idx, isin in enumerate(all_data.keys(), 1):
    print(f"\n[{idx}/{len(all_data)}] Creating consolidated tape for ISIN: {isin}")
    
    # Create consolidated tape for this ISIN
    tape = create_consolidated_tape_for_isin(isin, all_data)
    
    if not tape.empty:
        consolidated_tapes[isin] = tape
        
        # Display summary statistics
        num_venues = len([col for col in tape.columns if '_bid' in col])
        num_updates = len(tape)
        
        # Count non-null values to see data coverage
        bid_cols = [col for col in tape.columns if '_bid' in col]
        ask_cols = [col for col in tape.columns if '_ask' in col]
        
        total_bid_updates = tape[bid_cols].notna().sum().sum() if bid_cols else 0
        total_ask_updates = tape[ask_cols].notna().sum().sum() if ask_cols else 0
        
        print(f"  ✓ Created tape with {num_venues} venue(s)")
        print(f"  ✓ Total timestamps: {num_updates:,}")
        print(f"  ✓ Total bid updates: {total_bid_updates:,}")
        print(f"  ✓ Total ask updates: {total_ask_updates:,}")
        
        # Show first few rows as example
        if idx <= 3:  # Only show for first 3 ISINs to avoid clutter
            print(f"\n  Sample of consolidated tape (first 5 rows):")
            print(tape.head().to_string())
    else:
        print(f"  ⚠️  No data available for consolidated tape")

print("\n" + "=" * 70)
print(f"STEP 2 COMPLETE: Created consolidated tapes for {len(consolidated_tapes)} ISINs")
print("=" * 70)


In [ ]:
# ============================================================================
# VERIFICATION: Check Consolidated Tape Structure
# ============================================================================
# This verification demonstrates the consolidated tape structure using an
# ISIN that has multiple venues and uses the nanosecond trick (has duplicate
# epochs that were resolved).
# ============================================================================

if consolidated_tapes:
    # Find an ISIN with multiple venues (3-4) that also uses nanosecond trick
    # Market Microstructure: We want to demonstrate the consolidated tape
    # with a realistic example that shows:
    # 1. Multiple venues trading the same ISIN (fragmented market)
    # 2. Nanosecond trick usage (multiple snapshots at same microsecond)
    
    print("\n" + "=" * 70)
    print("VERIFICATION: Consolidated Tape Structure")
    print("=" * 70)
    
    # Search for an ISIN with multiple venues and nanosecond trick usage
    example_isin = None
    example_tape = None
    max_venues = 0
    has_nanosecond_trick = False
    
    # Priority: Find ISIN with 4 venues that uses nanosecond trick
    # Known examples from Step 1: ES0113900J37, ES0177542018 have 4 venues
    candidate_isins = ['ES0113900J37', 'ES0177542018', 'ES0113211835', 'ES0140609019']
    
    for candidate_isin in candidate_isins:
        if candidate_isin in consolidated_tapes:
            tape = consolidated_tapes[candidate_isin]
            num_venues = len([col for col in tape.columns if '_bid' in col])
            
            # Check if this ISIN uses nanosecond trick (has timestamps with nanosecond offsets)
            if candidate_isin in all_data:
                qte_dfs = all_data[candidate_isin].get('qte', {})
                for venue, qte_df in qte_dfs.items():
                    if not qte_df.empty:
                        # Check for nanosecond offsets (indicating nanosecond trick was used)
                        bursts = qte_df[qte_df.index.nanosecond > 0]
                        if not bursts.empty and num_venues >= 3:
                            example_isin = candidate_isin
                            example_tape = tape
                            max_venues = num_venues
                            has_nanosecond_trick = True
                            break
                if has_nanosecond_trick:
                    break
    
    # If no candidate found, find any ISIN with 3+ venues
    if example_isin is None:
        for isin, tape in consolidated_tapes.items():
            num_venues = len([col for col in tape.columns if '_bid' in col])
            if num_venues >= 3 and num_venues > max_venues:
                example_isin = isin
                example_tape = tape
                max_venues = num_venues
    
    # If still no candidate, use first available
    if example_isin is None and consolidated_tapes:
        example_isin = list(consolidated_tapes.keys())[0]
        example_tape = consolidated_tapes[example_isin]
        max_venues = len([col for col in example_tape.columns if '_bid' in col])
    
    if example_isin and example_tape is not None:
        print(f"\n📊 Example ISIN: {example_isin}")
        print(f"   Number of venues: {max_venues}")
        print(f"   Uses nanosecond trick: {has_nanosecond_trick}")
        
        # Get venue names
        bid_cols = [col for col in example_tape.columns if '_bid' in col]
        venues = [col.replace('_bid', '') for col in bid_cols]
        print(f"   Venues: {', '.join(venues)}")
        
        print(f"\n📐 Consolidated Tape Structure:")
        print(f"   Shape: {example_tape.shape} (rows, columns)")
        print(f"   Index Type: {type(example_tape.index).__name__}")
        print(f"   Index Unique: {example_tape.index.is_unique} ✓")
        print(f"   Total Timestamps: {len(example_tape):,}")
        
        print(f"\n📋 Column Names ({len(example_tape.columns)} total):")
        for col in sorted(example_tape.columns):
            non_null_count = example_tape[col].notna().sum()
            pct_coverage = (non_null_count / len(example_tape)) * 100
            print(f"   • {col:15s} ({non_null_count:>8,} values, {pct_coverage:>5.1f}% coverage)")
        
        print(f"\n📊 Sample Data (first 10 rows):")
        print(example_tape.head(10).to_string())
        
        # Demonstrate nanosecond trick if applicable
        if has_nanosecond_trick and example_isin in all_data:
            print(f"\n⏱️  Nanosecond Trick Demonstration:")
            qte_dfs = all_data[example_isin].get('qte', {})
            
            # Find a venue with nanosecond offsets
            for venue, qte_df in qte_dfs.items():
                if not qte_df.empty:
                    bursts = qte_df[qte_df.index.nanosecond > 0]
                    if not bursts.empty:
                        # Get first burst example
                        burst_ts = bursts.index[0]
                        base_ts = burst_ts.replace(nanosecond=0)
                        
                        # Get all snapshots at this base timestamp
                        burst_snapshots = qte_df.loc[base_ts : base_ts + pd.Timedelta(nanoseconds=999)]
                        
                        print(f"\n   Example from {venue} venue:")
                        print(f"   Base timestamp: {base_ts}")
                        print(f"   Number of snapshots at this microsecond: {len(burst_snapshots)}")
                        print(f"   These snapshots were resolved using nanosecond offsets:")
                        
                        # Show relevant columns
                        display_cols = ['epoch', 'px_bid_0', 'px_ask_0']
                        if 'sequence' in burst_snapshots.columns:
                            display_cols.insert(1, 'sequence')
                        
                        print(burst_snapshots[display_cols].to_string())
                        break
        
        print(f"\n💡 Market Microstructure Context:")
        print(f"   • Each row represents a point in time where at least one venue")
        print(f"     updated its orderbook")
        print(f"   • Forward fill (ffill) ensures that all venues' last known prices")
        print(f"     are available at every timestamp")
        print(f"   • This enables cross-venue price comparison for arbitrage detection")
        print(f"   • The nanosecond trick preserves chronological order when multiple")
        print(f"     snapshots occur at the same microsecond")
        
        # Show example of forward fill in action
        print(f"\n🔄 Forward Fill Example:")
        # Find a row where one venue has a value and another doesn't initially
        bid_cols = [col for col in example_tape.columns if '_bid' in col]
        if len(bid_cols) >= 2:
            # Find first non-null value for first venue
            first_venue_col = bid_cols[0]
            first_non_null_idx = example_tape[first_venue_col].first_valid_index()
            
            if first_non_null_idx is not None:
                # Show a few rows around this point
                start_idx = example_tape.index.get_loc(first_non_null_idx)
                end_idx = min(start_idx + 5, len(example_tape))
                sample_range = example_tape.iloc[start_idx:end_idx]
                
                print(f"   Showing how forward fill propagates prices:")
                print(f"   (First non-null value for {first_venue_col} at {first_non_null_idx})")
                print(sample_range[bid_cols].to_string())
    else:
        print("\n⚠️  No consolidated tapes available for verification.")
else:
    print("\n⚠️  No consolidated tapes created. Please check Step 1 execution.")


## **EXTRA** Visualization: Exploring Consolidated Tape Data

Before proceeding to Step 3 (Signal Generation), let's visualize the consolidated tape data to understand price dynamics, spreads, and potential arbitrage opportunities across venues. These visualizations will help us understand the market microstructure and identify patterns in the data.

### **(Estimado Francisco, en estos graficos a proposito he dejado la duplicidad de oportunidades de arbitraje puesto que todavia no hemos llegado al Step 3 en el cual nos instruias eliminarlos, los muestro a propósito porque creo que son muy ilustrativos a la hora de remarcar la importancia de lidiar con estas duplicidades ya que falsamente aumenta las oportunidades de arbitraje a cifras disparatadas y no reflejan lo que sucederia en la vida real, en el Step 3 lo corrijo, lo mismo sucede en el caso de los masivos outliers al comienzo y final de la sesión, a estos últimos les he dedicado un segmento muy largo antes de empezar el Step 3, ESPERO QUE TE GUSTE!)**

In [ ]:
# ============================================================================
# VISUALIZATION 1: Price Evolution Across Venues (Multi-Venue Comparison)
# ============================================================================
# This visualization shows how prices evolve across different venues for
# selected ISINs, similar to the XMAD vs CEUX comparison in the reference notebook.
# ============================================================================

# Select representative ISINs with multiple venues (3-4 venues preferred)
# Priority: ISINs with good data coverage and multiple venues
selected_isins = []

# Find ISINs with 3-4 venues
for isin, tape in consolidated_tapes.items():
    bid_cols = [col for col in tape.columns if '_bid' in col]
    num_venues = len(bid_cols)
    if num_venues >= 3:
        selected_isins.append((isin, num_venues))
        if len(selected_isins) >= 3:  # Select top 3 ISINs
            break

# Sort by number of venues (descending)
selected_isins.sort(key=lambda x: x[1], reverse=True)

print("=" * 70)
print("VISUALIZATION 1: Price Evolution Across Venues")
print("=" * 70)
print(f"\nSelected ISINs for visualization: {[isin for isin, _ in selected_isins]}")

# Create visualizations for selected ISINs
for isin, num_venues in selected_isins[:2]:  # Show first 2 ISINs
    if isin not in consolidated_tapes:
        continue
    
    tape = consolidated_tapes[isin]
    
    # Get venue names
    bid_cols = [col for col in tape.columns if '_bid' in col]
    ask_cols = [col for col in tape.columns if '_ask' in col]
    venues = [col.replace('_bid', '') for col in bid_cols]
    
    # Use full trading day data (from market open to market close)
    if len(tape) > 0:
        # Use complete data range - from first to last timestamp
        time_window = tape
        
        if len(time_window) > 0:
            # Create figure
            fig, axes = plt.subplots(2, 1, figsize=(16, 10))
            
            # Plot 1: Bid Prices
            ax1 = axes[0]
            for venue in venues:
                bid_col = f'{venue}_bid'
                if bid_col in time_window.columns:
                    ax1.plot(time_window.index, time_window[bid_col], 
                            label=f'{venue} Bid', linewidth=1.5, alpha=0.8)
            
            ax1.set_title(f'Bid Prices Across Venues - ISIN: {isin}', fontsize=14, fontweight='bold')
            ax1.set_ylabel('Bid Price (€)', fontsize=12)
            ax1.set_xlabel('Timestamp', fontsize=12)
            ax1.legend(loc='best', fontsize=10)
            ax1.grid(True, which='both', linestyle='--', linewidth=0.5, alpha=0.7)
            
            # Plot 2: Ask Prices
            ax2 = axes[1]
            for venue in venues:
                ask_col = f'{venue}_ask'
                if ask_col in time_window.columns:
                    ax2.plot(time_window.index, time_window[ask_col], 
                            label=f'{venue} Ask', linewidth=1.5, alpha=0.8, linestyle='--')
            
            ax2.set_title(f'Ask Prices Across Venues - ISIN: {isin}', fontsize=14, fontweight='bold')
            ax2.set_ylabel('Ask Price (€)', fontsize=12)
            ax2.set_xlabel('Timestamp', fontsize=12)
            ax2.legend(loc='best', fontsize=10)
            ax2.grid(True, which='both', linestyle='--', linewidth=0.5, alpha=0.7)
            
            plt.tight_layout()
            plt.show()
            
            print(f"\n✓ Created price evolution plots for {isin} ({num_venues} venues)")

print("\n" + "=" * 70)


In [ ]:
# ============================================================================
# VISUALIZATION 2: Spread Analysis (Price Differences Between Venues)
# ============================================================================
# This visualization shows price discrepancies between venues, similar to
# the spread plot in the reference notebook (XMAD - CEUX).
# ============================================================================

print("=" * 70)
print("VISUALIZATION 2: Spread Analysis Between Venues")
print("=" * 70)

# Use the same selected ISINs from Visualization 1
for isin, num_venues in selected_isins[:2]:  # Show first 2 ISINs
    if isin not in consolidated_tapes:
        continue
    
    tape = consolidated_tapes[isin]
    
    # Get venue names
    bid_cols = [col for col in tape.columns if '_bid' in col]
    venues = [col.replace('_bid', '') for col in bid_cols]
    
    if len(venues) < 2:
        continue
    
    # Use full trading day data (from market open to market close)
    time_window = tape
    
    if len(time_window) == 0:
        continue
    
    # Calculate spreads between venue pairs
    # Use BME as reference if available, otherwise use first venue
    reference_venue = 'BME' if 'BME' in venues else venues[0]
    
    # Create figure with subplots
    num_pairs = min(3, len(venues) - 1)  # Show up to 3 spread pairs
    fig, axes = plt.subplots(num_pairs, 1, figsize=(16, 4 * num_pairs))
    
    if num_pairs == 1:
        axes = [axes]
    
    plot_idx = 0
    for other_venue in venues:
        if other_venue == reference_venue or plot_idx >= num_pairs:
            continue
        
        # Calculate spread: Reference Bid - Other Ask (potential arbitrage direction)
        ref_bid_col = f'{reference_venue}_bid'
        other_ask_col = f'{other_venue}_ask'
        
        if ref_bid_col in time_window.columns and other_ask_col in time_window.columns:
            spread = time_window[ref_bid_col] - time_window[other_ask_col]
            
            ax = axes[plot_idx]
            ax.plot(time_window.index, spread, linewidth=1.5, alpha=0.8, color='steelblue')
            ax.axhline(y=0, color='red', linestyle='--', linewidth=1, alpha=0.5, label='Zero Spread')
            
            # Highlight positive spreads (potential arbitrage)
            positive_spread = spread[spread > 0]
            if len(positive_spread) > 0:
                ax.scatter(positive_spread.index, positive_spread.values, 
                          color='green', s=10, alpha=0.6, label='Positive Spread (Arbitrage Opportunity)')
            
            ax.set_title(f'Spread: {reference_venue} Bid - {other_venue} Ask | ISIN: {isin}', 
                        fontsize=12, fontweight='bold')
            ax.set_ylabel('Spread (€)', fontsize=11)
            ax.set_xlabel('Timestamp', fontsize=11)
            ax.legend(loc='best', fontsize=9)
            ax.grid(True, which='both', linestyle='--', linewidth=0.5, alpha=0.7)
            
            plot_idx += 1
    
    plt.tight_layout()
    plt.show()
    
    # Also create a combined spread visualization
    if len(venues) >= 2:
        fig, ax = plt.subplots(figsize=(16, 6))
        
        # Calculate all pairwise spreads and plot the maximum spread
        max_spreads = []
        min_spreads = []
        
        for i, venue1 in enumerate(venues):
            for venue2 in venues[i+1:]:
                bid_col1 = f'{venue1}_bid'
                ask_col2 = f'{venue2}_ask'
                bid_col2 = f'{venue2}_bid'
                ask_col1 = f'{venue1}_ask'
                
                if all(col in time_window.columns for col in [bid_col1, ask_col2, bid_col2, ask_col1]):
                    # Spread in both directions
                    spread1 = time_window[bid_col1] - time_window[ask_col2]
                    spread2 = time_window[bid_col2] - time_window[ask_col1]
                    
                    # Take the maximum spread at each timestamp
                    combined_spread = pd.concat([spread1, spread2], axis=1).max(axis=1)
                    max_spreads.append(combined_spread)
        
        if max_spreads:
            overall_max_spread = pd.concat(max_spreads, axis=1).max(axis=1)
            ax.plot(time_window.index, overall_max_spread, linewidth=1.5, alpha=0.8, 
                   color='purple', label='Maximum Cross-Venue Spread')
            ax.axhline(y=0, color='red', linestyle='--', linewidth=1, alpha=0.5)
            
            # Highlight positive regions
            positive_regions = overall_max_spread[overall_max_spread > 0]
            if len(positive_regions) > 0:
                ax.fill_between(positive_regions.index, 0, positive_regions.values, 
                              alpha=0.3, color='green', label='Arbitrage Opportunity Regions')
            
            ax.set_title(f'Maximum Cross-Venue Spread (Arbitrage Indicator) | ISIN: {isin}', 
                        fontsize=14, fontweight='bold')
            ax.set_ylabel('Spread (€)', fontsize=12)
            ax.set_xlabel('Timestamp', fontsize=12)
            ax.legend(loc='best', fontsize=10)
            ax.grid(True, which='both', linestyle='--', linewidth=0.5, alpha=0.7)
            
            plt.tight_layout()
            plt.show()
    
    print(f"\n✓ Created spread analysis plots for {isin}")

print("\n" + "=" * 70)


In [ ]:
# ============================================================================
# VISUALIZATION 3: Bid-Ask Spread Per Venue
# ============================================================================
# This visualization shows the bid-ask spread (liquidity cost) for each venue.
# Tighter spreads indicate better liquidity.
# ============================================================================

print("=" * 70)
print("VISUALIZATION 3: Bid-Ask Spread Per Venue")
print("=" * 70)

for isin, num_venues in selected_isins[:2]:  # Show first 2 ISINs
    if isin not in consolidated_tapes:
        continue
    
    tape = consolidated_tapes[isin]
    
    # Get venue names
    bid_cols = [col for col in tape.columns if '_bid' in col]
    venues = [col.replace('_bid', '') for col in bid_cols]
    
    # Use full trading day data (from market open to market close)
    time_window = tape
    
    if len(time_window) == 0:
        continue
    
    # Calculate bid-ask spread for each venue
    fig, axes = plt.subplots(len(venues), 1, figsize=(16, 4 * len(venues)))
    
    if len(venues) == 1:
        axes = [axes]
    
    for idx, venue in enumerate(venues):
        bid_col = f'{venue}_bid'
        ask_col = f'{venue}_ask'
        
        if bid_col in time_window.columns and ask_col in time_window.columns:
            # Calculate spread: Ask - Bid
            spread = time_window[ask_col] - time_window[bid_col]
            
            # Remove NaN values
            spread_clean = spread.dropna()
            
            if len(spread_clean) > 0:
                ax = axes[idx]
                ax.plot(spread_clean.index, spread_clean.values, 
                       linewidth=1.5, alpha=0.8, color='darkblue', label='Bid-Ask Spread')
                
                # Add mean line
                mean_spread = spread_clean.mean()
                ax.axhline(y=mean_spread, color='red', linestyle='--', 
                          linewidth=1.5, alpha=0.7, label=f'Mean Spread: {mean_spread:.4f} €')
                
                ax.set_title(f'{venue} Bid-Ask Spread | ISIN: {isin} | Mean: {mean_spread:.4f} €', 
                            fontsize=12, fontweight='bold')
                ax.set_ylabel('Spread (€)', fontsize=11)
                ax.set_xlabel('Timestamp', fontsize=11)
                ax.legend(loc='best', fontsize=9)
                ax.grid(True, which='both', linestyle='--', linewidth=0.5, alpha=0.7)
    
    plt.tight_layout()
    plt.show()
    
    # Create a comparison plot showing all venues together
    fig, ax = plt.subplots(figsize=(16, 6))
    
    for venue in venues:
        bid_col = f'{venue}_bid'
        ask_col = f'{venue}_ask'
        
        if bid_col in time_window.columns and ask_col in time_window.columns:
            spread = time_window[ask_col] - time_window[bid_col]
            spread_clean = spread.dropna()
            
            if len(spread_clean) > 0:
                ax.plot(spread_clean.index, spread_clean.values, 
                       linewidth=1.5, alpha=0.7, label=f'{venue} (Mean: {spread_clean.mean():.4f} €)')
    
    ax.set_title(f'Bid-Ask Spread Comparison Across Venues | ISIN: {isin}', 
                fontsize=14, fontweight='bold')
    ax.set_ylabel('Bid-Ask Spread (€)', fontsize=12)
    ax.set_xlabel('Timestamp', fontsize=12)
    ax.legend(loc='best', fontsize=10)
    ax.grid(True, which='both', linestyle='--', linewidth=0.5, alpha=0.7)
    
    plt.tight_layout()
    plt.show()
    
    print(f"\n✓ Created bid-ask spread plots for {isin}")

print("\n" + "=" * 70)


In [ ]:
# ============================================================================
# VISUALIZATION 4: Arbitrage Opportunity Visualization
# ============================================================================
# This visualization identifies where arbitrage opportunities exist by showing
# max_bid (across all venues) vs min_ask (across all venues).
# When max_bid > min_ask, an arbitrage opportunity exists.
# ============================================================================

print("=" * 70)
print("VISUALIZATION 4: Arbitrage Opportunity Detection")
print("=" * 70)

# Find ISINs that actually have arbitrage opportunities
arbitrage_isins = []

for isin, tape in consolidated_tapes.items():
    bid_cols = [col for col in tape.columns if '_bid' in col]
    ask_cols = [col for col in tape.columns if '_ask' in col]
    
    if len(bid_cols) < 2:  # Need at least 2 venues for arbitrage
        continue
    
    # Calculate max bid and min ask across all venues
    max_bid = tape[bid_cols].max(axis=1)
    min_ask = tape[ask_cols].min(axis=1)
    
    # Check if there are any arbitrage opportunities
    theoretical_profit = max_bid - min_ask
    arbitrage_opportunity = theoretical_profit > 0
    
    if arbitrage_opportunity.sum() > 0:  # Has at least some arbitrage opportunities
        num_opportunities = arbitrage_opportunity.sum()
        max_profit = theoretical_profit.max()
        arbitrage_isins.append((isin, len(bid_cols), num_opportunities, max_profit))

# Sort by number of opportunities (descending), then by max profit
arbitrage_isins.sort(key=lambda x: (x[2], x[3]), reverse=True)

print(f"\nFound {len(arbitrage_isins)} ISINs with arbitrage opportunities")
if len(arbitrage_isins) > 0:
    print(f"Top ISINs with arbitrage opportunities:")
    for isin, num_venues, num_opps, max_prof in arbitrage_isins[:5]:
        print(f"  {isin}: {num_opps:,} opportunities, max profit: {max_prof:.4f} €")

# Use ISINs with arbitrage opportunities, or fall back to selected_isins if none found
if len(arbitrage_isins) > 0:
    display_isins = arbitrage_isins[:2]  # Show top 2 ISINs with arbitrage
else:
    print("\n⚠️  No ISINs with arbitrage opportunities found. Using selected ISINs instead.")
    display_isins = [(isin, num_venues, 0, 0) for isin, num_venues in selected_isins[:2]]

for isin_info in display_isins:
    isin = isin_info[0]
    num_venues = isin_info[1]
    
    if isin not in consolidated_tapes:
        continue
    
    tape = consolidated_tapes[isin]
    
    # Get all bid and ask columns
    bid_cols = [col for col in tape.columns if '_bid' in col]
    ask_cols = [col for col in tape.columns if '_ask' in col]
    
    if len(bid_cols) < 2:  # Need at least 2 venues for arbitrage
        continue
    
    # Use full trading day data (from market open to market close)
    time_window = tape
    
    if len(time_window) == 0:
        continue
    
    # Calculate max bid and min ask across all venues at each timestamp
    max_bid = time_window[bid_cols].max(axis=1)
    min_ask = time_window[ask_cols].min(axis=1)
    
    # Calculate theoretical profit (arbitrage opportunity)
    # Profit exists when max_bid > min_ask
    theoretical_profit = max_bid - min_ask
    arbitrage_opportunity = theoretical_profit > 0
    
    # Create main visualization
    fig, axes = plt.subplots(3, 1, figsize=(16, 12))
    
    # Plot 1: Max Bid vs Min Ask
    ax1 = axes[0]
    ax1.plot(time_window.index, max_bid, label='Max Bid (across all venues)', 
            linewidth=2, alpha=0.9, color='green')
    ax1.plot(time_window.index, min_ask, label='Min Ask (across all venues)', 
            linewidth=2, alpha=0.9, color='red', linestyle='--')
    
    # Highlight arbitrage opportunities (where max_bid > min_ask)
    arb_mask = arbitrage_opportunity
    if arb_mask.sum() > 0:
        ax1.fill_between(time_window.index, max_bid, min_ask, 
                         where=arb_mask, alpha=0.3, color='yellow', 
                         label='Arbitrage Opportunity Region')
    
    ax1.set_title(f'Arbitrage Opportunity Detection: Max Bid vs Min Ask | ISIN: {isin}', 
                 fontsize=14, fontweight='bold')
    ax1.set_ylabel('Price (€)', fontsize=12)
    ax1.legend(loc='best', fontsize=10)
    ax1.grid(True, which='both', linestyle='--', linewidth=0.5, alpha=0.7)
    
    # Plot 2: Theoretical Profit
    ax2 = axes[1]
    ax2.plot(time_window.index, theoretical_profit, 
            linewidth=1.5, alpha=0.8, color='purple', label='Theoretical Profit')
    ax2.axhline(y=0, color='black', linestyle='-', linewidth=1, alpha=0.5)
    
    # Highlight positive profit regions
    positive_profit = theoretical_profit[theoretical_profit > 0]
    if len(positive_profit) > 0:
        ax2.fill_between(positive_profit.index, 0, positive_profit.values, 
                        alpha=0.4, color='green', label='Profitable Opportunities')
    
    ax2.set_title(f'Theoretical Profit per Unit (Max Bid - Min Ask) | ISIN: {isin}', 
                 fontsize=14, fontweight='bold')
    ax2.set_ylabel('Profit (€)', fontsize=12)
    ax2.legend(loc='best', fontsize=10)
    ax2.grid(True, which='both', linestyle='--', linewidth=0.5, alpha=0.7)
    
    # Plot 3: Arbitrage Opportunity Indicator (Binary)
    ax3 = axes[2]
    ax3.fill_between(time_window.index, 0, arbitrage_opportunity.astype(int), 
                    alpha=0.6, color='green', label='Arbitrage Opportunity Present')
    ax3.set_title(f'Arbitrage Opportunity Indicator (1 = Opportunity Exists) | ISIN: {isin}', 
                 fontsize=14, fontweight='bold')
    ax3.set_ylabel('Opportunity (0/1)', fontsize=12)
    ax3.set_xlabel('Timestamp', fontsize=12)
    ax3.set_ylim(-0.1, 1.1)
    ax3.legend(loc='best', fontsize=10)
    ax3.grid(True, which='both', linestyle='--', linewidth=0.5, alpha=0.7)
    
    plt.tight_layout()
    plt.show()
    
    # Print statistics
    num_opportunities = arbitrage_opportunity.sum()
    total_timestamps = len(arbitrage_opportunity)
    opportunity_pct = (num_opportunities / total_timestamps) * 100 if total_timestamps > 0 else 0
    
    if num_opportunities > 0:
        max_profit = theoretical_profit.max()
        mean_profit = positive_profit.mean() if len(positive_profit) > 0 else 0
        
        print(f"\n📊 Arbitrage Statistics for {isin}:")
        print(f"   • Total timestamps analyzed: {total_timestamps:,}")
        print(f"   • Arbitrage opportunities found: {num_opportunities:,} ({opportunity_pct:.2f}%)")
        print(f"   • Maximum theoretical profit: {max_profit:.4f} €")
        print(f"   • Mean profit (when opportunity exists): {mean_profit:.4f} €")
    else:
        print(f"\n⚠️  No arbitrage opportunities found for {isin} in the selected time window")
    
    print(f"\n✓ Created arbitrage opportunity plots for {isin}")

print("\n" + "=" * 70)


In [ ]:
# ============================================================================
# VISUALIZATION 5: Summary Statistics Visualization
# ============================================================================
# This visualization provides an overview of key statistics about the
# consolidated tape data using bar charts and summary metrics.
# ============================================================================

print("=" * 70)
print("VISUALIZATION 5: Summary Statistics Visualization")
print("=" * 70)

# Collect statistics across all ISINs
venue_stats = {}
isin_stats = []

for isin, tape in consolidated_tapes.items():
    bid_cols = [col for col in tape.columns if '_bid' in col]
    ask_cols = [col for col in tape.columns if '_ask' in col]
    venues = [col.replace('_bid', '') for col in bid_cols]
    
    # Count updates per venue
    for venue in venues:
        bid_col = f'{venue}_bid'
        ask_col = f'{venue}_ask'
        
        if venue not in venue_stats:
            venue_stats[venue] = {'updates': 0, 'isins': set()}
        
        if bid_col in tape.columns:
            venue_stats[venue]['updates'] += tape[bid_col].notna().sum()
        if ask_col in tape.columns:
            venue_stats[venue]['updates'] += tape[ask_col].notna().sum()
        
        venue_stats[venue]['isins'].add(isin)
    
    # Calculate statistics for this ISIN
    if len(bid_cols) > 0 and len(ask_cols) > 0:
        # Calculate price ranges
        all_bids = []
        all_asks = []
        for bid_col in bid_cols:
            all_bids.extend(tape[bid_col].dropna().tolist())
        for ask_col in ask_cols:
            all_asks.extend(tape[ask_col].dropna().tolist())
        
        if len(all_bids) > 0 and len(all_asks) > 0:
            min_bid = min(all_bids)
            max_bid = max(all_bids)
            min_ask = min(all_asks)
            max_ask = max(all_asks)
            
            # Calculate average bid-ask spread
            spreads = []
            for venue in venues:
                bid_col = f'{venue}_bid'
                ask_col = f'{venue}_ask'
                if bid_col in tape.columns and ask_col in tape.columns:
                    spread = tape[ask_col] - tape[bid_col]
                    spreads.extend(spread.dropna().tolist())
            
            avg_spread = np.mean(spreads) if len(spreads) > 0 else 0
            
            isin_stats.append({
                'ISIN': isin,
                'Num_Venues': len(venues),
                'Total_Updates': len(tape),
                'Min_Bid': min_bid,
                'Max_Bid': max_bid,
                'Min_Ask': min_ask,
                'Max_Ask': max_ask,
                'Avg_Spread': avg_spread
            })

# Create visualizations
fig = plt.figure(figsize=(18, 12))

# Plot 1: Updates per Venue (Bar Chart)
ax1 = plt.subplot(2, 3, 1)
venues_list = list(venue_stats.keys())
updates_list = [venue_stats[v]['updates'] for v in venues_list]
colors = plt.cm.Set3(np.linspace(0, 1, len(venues_list)))

bars = ax1.bar(venues_list, updates_list, color=colors, alpha=0.8, edgecolor='black', linewidth=1.5)
ax1.set_title('Total Updates per Venue (All ISINs)', fontsize=12, fontweight='bold')
ax1.set_ylabel('Number of Updates', fontsize=11)
ax1.set_xlabel('Venue', fontsize=11)
ax1.grid(True, axis='y', linestyle='--', linewidth=0.5, alpha=0.7)

# Add value labels on bars
for bar in bars:
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height,
            f'{int(height):,}',
            ha='center', va='bottom', fontsize=9)

# Plot 2: Number of ISINs per Venue
ax2 = plt.subplot(2, 3, 2)
isins_per_venue = [len(venue_stats[v]['isins']) for v in venues_list]
bars2 = ax2.bar(venues_list, isins_per_venue, color=colors, alpha=0.8, edgecolor='black', linewidth=1.5)
ax2.set_title('Number of ISINs per Venue', fontsize=12, fontweight='bold')
ax2.set_ylabel('Number of ISINs', fontsize=11)
ax2.set_xlabel('Venue', fontsize=11)
ax2.grid(True, axis='y', linestyle='--', linewidth=0.5, alpha=0.7)

for bar in bars2:
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height,
            f'{int(height)}',
            ha='center', va='bottom', fontsize=9)

# Plot 3: Distribution of Number of Venues per ISIN
ax3 = plt.subplot(2, 3, 3)
if isin_stats:
    df_isin_stats = pd.DataFrame(isin_stats)
    venue_counts = df_isin_stats['Num_Venues'].value_counts().sort_index()
    bars3 = ax3.bar(venue_counts.index.astype(str), venue_counts.values, 
                   color='steelblue', alpha=0.8, edgecolor='black', linewidth=1.5)
    ax3.set_title('Distribution: Number of Venues per ISIN', fontsize=12, fontweight='bold')
    ax3.set_ylabel('Number of ISINs', fontsize=11)
    ax3.set_xlabel('Number of Venues', fontsize=11)
    ax3.grid(True, axis='y', linestyle='--', linewidth=0.5, alpha=0.7)
    
    for bar in bars3:
        height = bar.get_height()
        ax3.text(bar.get_x() + bar.get_width()/2., height,
                f'{int(height)}',
                ha='center', va='bottom', fontsize=9)

# Plot 4: Average Bid-Ask Spread by ISIN (Top 10)
ax4 = plt.subplot(2, 3, 4)
if isin_stats:
    df_isin_stats = pd.DataFrame(isin_stats)
    top_spreads = df_isin_stats.nlargest(10, 'Avg_Spread')
    bars4 = ax4.barh(range(len(top_spreads)), top_spreads['Avg_Spread'], 
                    color='coral', alpha=0.8, edgecolor='black', linewidth=1.5)
    ax4.set_yticks(range(len(top_spreads)))
    ax4.set_yticklabels([isin[:15] + '...' if len(isin) > 15 else isin 
                        for isin in top_spreads['ISIN']], fontsize=8)
    ax4.set_title('Top 10 ISINs by Average Bid-Ask Spread', fontsize=12, fontweight='bold')
    ax4.set_xlabel('Average Spread (€)', fontsize=11)
    ax4.grid(True, axis='x', linestyle='--', linewidth=0.5, alpha=0.7)

# Plot 5: Total Updates Distribution
ax5 = plt.subplot(2, 3, 5)
if isin_stats:
    df_isin_stats = pd.DataFrame(isin_stats)
    ax5.hist(df_isin_stats['Total_Updates'], bins=30, color='mediumseagreen', 
            alpha=0.8, edgecolor='black', linewidth=1.5)
    ax5.set_title('Distribution of Total Updates per ISIN', fontsize=12, fontweight='bold')
    ax5.set_ylabel('Number of ISINs', fontsize=11)
    ax5.set_xlabel('Total Updates', fontsize=11)
    ax5.grid(True, axis='y', linestyle='--', linewidth=0.5, alpha=0.7)

# Plot 6: Price Range Statistics
ax6 = plt.subplot(2, 3, 6)
if isin_stats:
    df_isin_stats = pd.DataFrame(isin_stats)
    # Calculate price range (max - min) for bids and asks
    df_isin_stats['Bid_Range'] = df_isin_stats['Max_Bid'] - df_isin_stats['Min_Bid']
    df_isin_stats['Ask_Range'] = df_isin_stats['Max_Ask'] - df_isin_stats['Min_Ask']
    
    x_pos = np.arange(len(df_isin_stats))
    width = 0.35
    
    # Show top 10 ISINs by bid range
    top_ranges = df_isin_stats.nlargest(10, 'Bid_Range')
    x_pos = np.arange(len(top_ranges))
    
    bars6a = ax6.barh(x_pos - width/2, top_ranges['Bid_Range'], width, 
                     label='Bid Range', color='lightblue', alpha=0.8, edgecolor='black')
    bars6b = ax6.barh(x_pos + width/2, top_ranges['Ask_Range'], width, 
                     label='Ask Range', color='lightcoral', alpha=0.8, edgecolor='black')
    
    ax6.set_yticks(x_pos)
    ax6.set_yticklabels([isin[:12] + '...' if len(isin) > 12 else isin 
                         for isin in top_ranges['ISIN']], fontsize=8)
    ax6.set_title('Top 10 ISINs by Price Range', fontsize=12, fontweight='bold')
    ax6.set_xlabel('Price Range (€)', fontsize=11)
    ax6.legend(loc='best', fontsize=9)
    ax6.grid(True, axis='x', linestyle='--', linewidth=0.5, alpha=0.7)

plt.tight_layout()
plt.show()

# Print summary statistics
print("\n📊 Overall Summary Statistics:")
print(f"   • Total ISINs analyzed: {len(consolidated_tapes)}")
print(f"   • Total venues: {len(venue_stats)}")
print(f"   • Venues: {', '.join(venues_list)}")

print("\n📈 Updates per Venue:")
for venue in venues_list:
    print(f"   • {venue}: {venue_stats[venue]['updates']:,} updates across {len(venue_stats[venue]['isins'])} ISINs")

if isin_stats:
    df_isin_stats = pd.DataFrame(isin_stats)
    print(f"\n📊 ISIN Statistics:")
    print(f"   • Average number of venues per ISIN: {df_isin_stats['Num_Venues'].mean():.2f}")
    print(f"   • Average total updates per ISIN: {df_isin_stats['Total_Updates'].mean():.0f}")
    print(f"   • Average bid-ask spread: {df_isin_stats['Avg_Spread'].mean():.4f} €")

print("\n" + "=" * 70)
print("✓ Created summary statistics visualizations")
print("=" * 70)


### Step 3: Signal Generation

- **Arbitrage Condition:** An opportunity exists when Global Max Bid > Global Min Ask.
- **Profit Calc:** (Max Bid - Min Ask) * Min(BidQty, AskQty).
- **Rising Edge:** In a simulation, if an opportunity persists for 1 second (1000 snapshots), you can only trade it *once* (the first time it appears). Ensure you aren't "double counting" the same opportunity. If the opportunity vanishes and quickly reappears you can count it as a new opportunity for simplification.
- **Simplification:** Only look at opportunities between Global Max Bid and Global Min Ask. There might be others at the second or third price levels of the orderbook, but let's make it simple and use only the best Bid Ask of each trading venue.

### Edge Cases in Signal Generation: Why They Appear and How We Handle Them

As we observed in the **visualizations from Step 2** (particularly in the price evolution and spread analysis plots), the consolidated tape contains several edge cases that must be carefully handled to ensure accurate arbitrage signal generation:

#### 1. **Missing Data (NaN Values) in Consolidated Tape**

**Why They Appear:**
- **Market Opening/Closing:** At the start of the trading day, venues open at different times. For example, BME might open at 08:00:00, while CBOE opens at 08:00:05. During this 5-microsecond gap, CBOE has no data, resulting in NaN values.
- **Asynchronous Updates:** Even during continuous trading, venues update their orderbooks independently. If BME updates at t=100µs and CBOE hasn't updated since t=95µs, forward fill propagates CBOE's last known price. However, if a venue has never updated yet (e.g., at market open), we get NaN.
- **Venue-Specific Trading:** Some ISINs may not trade on all venues. For example, a Spanish stock might trade on BME and CBOE but not on TURQUOISE, leading to permanent NaN values for that venue.

**Why We Must Deal With Them:**
- **`.idxmax()` and `.idxmin()` Failures:** When calculating which venue has the maximum bid or minimum ask, if all values are NaN at a timestamp, `idxmax()`/`idxmin()` return NaN. Attempting to extract the venue name from NaN using string operations would cause errors.
- **False Arbitrage Signals:** If we don't filter NaN values properly, we might calculate `NaN > 10.50` which evaluates to False, but then try to extract quantities from a NaN venue name, causing crashes.
- **Incorrect Profit Calculations:** Missing quantities would lead to NaN profits, which would skew our total profit calculations.

**How We Handle It:**
- We use `.fillna('')` before string operations to safely handle NaN venue names
- We check for empty consolidated tapes and missing quantity columns before processing
- We filter out opportunities where critical data (prices or quantities) is missing
- We use `.fillna(False)` in rising edge detection to handle the first timestamp edge case

#### 2. **Empty or Incomplete Consolidated Tapes**

**Why They Appear:**
- Some ISINs may have very limited trading activity
- Data quality issues during cleaning (all prices filtered out as invalid)
- ISINs that only trade on one venue (no arbitrage possible anyway)

**Why We Must Deal With Them:**
- Calling `.max(axis=1)` on an empty DataFrame would fail
- Processing empty data wastes computational resources

**How We Handle It:**
- Early return with empty DataFrame if consolidated tape is empty
- Check for presence of required columns (bid_cols, ask_cols) before operations

#### 3. **Missing Quantities in Consolidated Tape**

**Why They Appear:**
- The original consolidated tape from Step 2 only includes prices, not quantities
- Quantities are needed for profit calculation but weren't included initially

**Why We Must Deal With Them:**
- Without quantities, we cannot calculate `Min(BidQty, AskQty)` for profit calculation
- We need to know how much volume is available at the arbitrage prices

**How We Handle It:**
- We check if quantities exist in the tape, and if not, we extend it using `extend_consolidated_tape_with_quantities()`
- This function follows the same forward-fill pattern as prices to maintain consistency

#### 4. **Rising Edge Detection Edge Cases**

**Why They Appear:**
- At the very first timestamp, there is no "previous" state to compare against
- Opportunities that persist for extended periods (as seen in the visualizations)

**Why We Must Deal With Them:**
- `.shift(1)` on the first row returns NaN, which would break boolean operations
- We need to correctly identify the first appearance of an opportunity

**How We Handle It:**
- We use `.fillna(False)` to treat the "before first timestamp" state as "no opportunity"
- This ensures the first opportunity (if it exists) is correctly identified as a rising edge

#### 5. **Venue Mismatches Between Prices and Quantities**

**Why They Appear:**
- Different venues may have different update frequencies
- A venue might have a price update but the corresponding quantity column might be missing or NaN

**Why We Must Deal With Them:**
- We need to extract quantities from the specific venue that has the max bid or min ask
- If the quantity column doesn't exist for that venue, we get NaN, which would break profit calculations

**How We Handle It:**
- We check for the existence of quantity columns before extraction
- We initialize quantity Series with NaN and only fill them when data is available
- The `Min(BidQty, AskQty)` operation naturally handles NaN by returning NaN, which we then filter out

---

**Key Insight from Step 2 Visualizations:**
The price evolution plots showed that venues often have gaps in their data, especially at market open/close. The spread analysis revealed that arbitrage opportunities appear and disappear frequently. These visual patterns directly inform our edge case handling strategy, ensuring our signal generation is robust to real-world market data imperfections.


In [ ]:
# ============================================================================
# STEP 3: SIGNAL GENERATION
# ============================================================================
# This step generates arbitrage signals by:
# 1. Extending consolidated tape with quantities (if not already present)
# 2. Calculating Global Max Bid and Global Min Ask
# 3. Identifying arbitrage opportunities (Max Bid > Min Ask)
# 4. Calculating profit per opportunity
# 5. Implementing rising edge detection to avoid double-counting
# ============================================================================

def extend_consolidated_tape_with_quantities(isin, consolidated_tape, cleaned_data_dict):
    """
    Extends the consolidated tape to include bid and ask quantities.
    
    The consolidated tape from Step 2 only includes prices. This function
    adds quantities following the same forward-fill pattern as prices.
    
    Market Microstructure Context:
    - Quantities are needed to calculate tradable volume for arbitrage
    - We forward-fill quantities just like prices to maintain consistency
    - This ensures we know available liquidity at each timestamp
    
    Args:
        isin: The ISIN identifier
        consolidated_tape: DataFrame with prices (from Step 2)
        cleaned_data_dict: Dictionary with original cleaned QTE data
    
    Returns:
        DataFrame with prices and quantities (bid_qty, ask_qty for each venue)
    """
    # ========================================================================
    # EDGE CASE HANDLING 1: Empty Consolidated Tape
    # ========================================================================
    # Market Microstructure: Some ISINs may have no data after cleaning,
    # or may only trade on one venue (no arbitrage possible). We must check
    # for empty tapes before processing to avoid errors.
    if consolidated_tape.empty:
        return consolidated_tape
    
    # EDGE CASE HANDLING 2: Missing ISIN in Data Dictionary
    # If the ISIN is not in the cleaned data dictionary, we cannot extend
    # the tape with quantities. Return the original tape.
    if isin not in cleaned_data_dict:
        return consolidated_tape
    
    qte_dfs = cleaned_data_dict[isin].get('qte', {})
    
    if not qte_dfs:
        return consolidated_tape
    
    # ========================================================================
    # Extract quantities from each venue (same pattern as prices)
    # ========================================================================
    venue_data_list = []
    
    for venue, qte_df in qte_dfs.items():
        if qte_df.empty:
            continue
        
        # Extract quantities
        venue_subset = qte_df[['qty_bid_0', 'qty_ask_0']].copy()
        venue_subset['venue'] = venue
        
        # Rename columns
        venue_subset = venue_subset.rename(columns={
            'qty_bid_0': 'bid_qty',
            'qty_ask_0': 'ask_qty'
        })
        
        venue_data_list.append(venue_subset)
    
    if not venue_data_list:
        return consolidated_tape
    
    # Combine all venues
    tape_full = pd.concat(venue_data_list)
    tape_full.sort_index(inplace=True)
    
    # Pivot quantities separately
    tape_bid_qty_pivot = tape_full.pivot(columns='venue', values='bid_qty')
    tape_ask_qty_pivot = tape_full.pivot(columns='venue', values='ask_qty')
    
    # Forward fill quantities (same as prices)
    tape_bid_qty_ffill = tape_bid_qty_pivot.ffill()
    tape_ask_qty_ffill = tape_ask_qty_pivot.ffill()
    
    # Rename columns to include suffix
    tape_bid_qty_ffill.columns = [f'{venue}_bid_qty' for venue in tape_bid_qty_ffill.columns]
    tape_ask_qty_ffill.columns = [f'{venue}_ask_qty' for venue in tape_ask_qty_ffill.columns]
    
    # Combine with existing consolidated tape
    extended_tape = pd.concat([consolidated_tape, tape_bid_qty_ffill, tape_ask_qty_ffill], axis=1)
    extended_tape = extended_tape.sort_index(axis=1)
    
    return extended_tape


def generate_arbitrage_signals_for_isin(isin, consolidated_tape, cleaned_data_dict):
    """
    Generates arbitrage signals for a single ISIN.
    
    This function identifies arbitrage opportunities where Global Max Bid > Global Min Ask,
    calculates profits, and implements rising edge detection to avoid double-counting.
    
    Market Microstructure Context:
    - Arbitrage exists when we can buy at the lowest ask and sell at the highest bid
    - We can only trade the minimum of available bid and ask quantities
    - Rising edge ensures we don't count the same opportunity multiple times
    - Once an opportunity vanishes, we can trade it again when it reappears
    
    Args:
        isin: The ISIN identifier
        consolidated_tape: DataFrame with prices (and optionally quantities)
        cleaned_data_dict: Dictionary with original cleaned data (for quantities if needed)
    
    Returns:
        DataFrame with arbitrage signals (only rising edges) containing:
        - timestamp, prices, quantities, venues, profits
    """
    # ========================================================================
    # EDGE CASE HANDLING 1: Empty Consolidated Tape
    # ========================================================================
    # Market Microstructure: Some ISINs may have no valid data after cleaning
    # (e.g., all prices filtered as invalid magic numbers). We must check
    # before processing to avoid errors in vectorized operations.
    if consolidated_tape.empty:
        return pd.DataFrame()
    
    # ========================================================================
    # STEP 1: Extend consolidated tape with quantities if not present
    # ========================================================================
    # EDGE CASE HANDLING 2: Missing Quantities in Consolidated Tape
    # The consolidated tape from Step 2 only includes prices. We need quantities
    # to calculate tradable volume. This check ensures we extend the tape if needed.
    # Market Microstructure: Quantities are essential for profit calculation as
    # we can only trade Min(BidQty, AskQty) - the maximum executable volume.
    bid_qty_cols = [col for col in consolidated_tape.columns if '_bid_qty' in col]
    ask_qty_cols = [col for col in consolidated_tape.columns if '_ask_qty' in col]
    
    if not bid_qty_cols or not ask_qty_cols:
        # Extend tape with quantities
        consolidated_tape = extend_consolidated_tape_with_quantities(
            isin, consolidated_tape, cleaned_data_dict
        )
        # Re-extract column lists after extension
        bid_qty_cols = [col for col in consolidated_tape.columns if '_bid_qty' in col]
        ask_qty_cols = [col for col in consolidated_tape.columns if '_ask_qty' in col]
    
    # Get bid and ask price columns (exclude quantity columns)
    bid_cols = [col for col in consolidated_tape.columns if '_bid' in col and '_qty' not in col]
    ask_cols = [col for col in consolidated_tape.columns if '_ask' in col and '_qty' not in col]
    
    # EDGE CASE HANDLING 3: Missing Price Columns
    # If no bid or ask price columns exist, we cannot calculate arbitrage.
    # This can happen if data structure is corrupted or ISIN has no valid prices.
    if not bid_cols or not ask_cols:
        return pd.DataFrame()
    
    # ========================================================================
    # STEP 2: Calculate Global Max Bid and Global Min Ask
    # ========================================================================
    # Market Microstructure: Global Max Bid is the highest price any venue
    # is willing to pay. Global Min Ask is the lowest price any venue is
    # willing to sell. When Max Bid > Min Ask, arbitrage is possible.
    
    global_max_bid = consolidated_tape[bid_cols].max(axis=1)
    global_min_ask = consolidated_tape[ask_cols].min(axis=1)
    
    # ========================================================================
    # STEP 3: Identify Arbitrage Opportunities
    # ========================================================================
    # An opportunity exists when we can buy at Min Ask and sell at Max Bid
    # simultaneously, making a risk-free profit.
    
    arbitrage_opportunity = global_max_bid > global_min_ask
    
    # ========================================================================
    # STEP 4: Identify Venues with Max Bid and Min Ask
    # ========================================================================
    # We need to know which venues to trade on and what quantities are available
    
    # ========================================================================
    # EDGE CASE HANDLING 4: NaN Values from idxmax/idxmin
    # ========================================================================
    # Market Microstructure: As seen in Step 2 visualizations, venues open
    # at different times and update asynchronously. At market open, some venues
    # may have no data yet, resulting in all-NaN rows. When all bids (or asks)
    # are NaN, idxmax()/idxmin() returns NaN, which would cause errors in
    # string operations.
    #
    # Why this happens:
    # - Market opening: BME opens at 08:00:00, CBOE at 08:00:05 → NaN for CBOE
    # - Venue-specific trading: Some ISINs don't trade on all venues → permanent NaN
    # - Data gaps: Forward fill can't fill if venue never had data
    #
    # How we handle it:
    # - Use fillna('') before string operations to safely handle NaN
    # - Convert empty strings back to NaN for consistency
    # - This ensures we can extract venue names even when some timestamps have NaN
    
    # Find which venue has the max bid at each timestamp
    max_bid_venue_idx = consolidated_tape[bid_cols].idxmax(axis=1)
    # Extract venue name from column name (e.g., 'BME_bid' -> 'BME')
    # CRITICAL: Handle NaN values (when all bids are NaN at a timestamp)
    max_bid_venue = max_bid_venue_idx.fillna('').str.replace('_bid', '', regex=False)
    max_bid_venue = max_bid_venue.replace('', np.nan)
    
    # Find which venue has the min ask at each timestamp
    min_ask_venue_idx = consolidated_tape[ask_cols].idxmin(axis=1)
    # Extract venue name from column name (e.g., 'BME_ask' -> 'BME')
    # CRITICAL: Handle NaN values (when all asks are NaN at a timestamp)
    min_ask_venue = min_ask_venue_idx.fillna('').str.replace('_ask', '', regex=False)
    min_ask_venue = min_ask_venue.replace('', np.nan)
    
    # ========================================================================
    # STEP 5: Extract Quantities for Max Bid and Min Ask Venues
    # ========================================================================
    # Market Microstructure: We can only trade the minimum of available
    # bid and ask quantities. This represents the maximum executable volume.
    #
    # ========================================================================
    # EDGE CASE HANDLING 5: Missing Quantity Columns for Specific Venues
    # ========================================================================
    # Market Microstructure: As seen in Step 2 visualizations, different venues
    # update at different frequencies. A venue might have price data but missing
    # quantity data, or the quantity column might not exist for that venue.
    #
    # Why this happens:
    # - Different update frequencies: Price updates more frequently than quantity
    # - Data quality issues: Quantity data might be filtered out during cleaning
    # - Venue-specific data structure: Some venues might not report quantities
    # - Market microstructure: At market open, quantities may not be available yet
    #
    # Why we must deal with it:
    # - KeyError: If we try to access a non-existent column, pandas raises KeyError
    # - Incorrect profits: Missing quantities would lead to NaN profits
    # - False signals: We need quantities to calculate tradable volume
    #
    # How we handle it:
    # - Initialize Series with NaN (safe default for missing data)
    # - Check for column existence before extraction (if bid_qty_col in columns)
    # - Skip NaN venues (pd.isna(venue) check)
    # - NaN quantities will result in NaN profits, which are naturally filtered
    #   out when we only keep opportunities with valid data
    
    # Initialize quantity Series with NaN (safe default for missing data)
    bid_qty = pd.Series(index=consolidated_tape.index, dtype=float)
    ask_qty = pd.Series(index=consolidated_tape.index, dtype=float)
    
    # For each venue, extract quantities when it's the max bid or min ask venue
    # CRITICAL: Check for NaN venues and missing columns to avoid errors
    for venue in max_bid_venue.unique():
        if pd.isna(venue):  # Skip NaN venues (from edge case handling 4)
            continue
        bid_qty_col = f'{venue}_bid_qty'
        if bid_qty_col in consolidated_tape.columns:  # Check column exists
            mask = max_bid_venue == venue
            bid_qty.loc[mask] = consolidated_tape.loc[mask, bid_qty_col]
    
    for venue in min_ask_venue.unique():
        if pd.isna(venue):  # Skip NaN venues (from edge case handling 4)
            continue
        ask_qty_col = f'{venue}_ask_qty'
        if ask_qty_col in consolidated_tape.columns:  # Check column exists
            mask = min_ask_venue == venue
            ask_qty.loc[mask] = consolidated_tape.loc[mask, ask_qty_col]
    
    # ========================================================================
    # STEP 6: Calculate Profit per Opportunity
    # ========================================================================
    # Profit per unit = Max Bid - Min Ask
    # Tradable quantity = Min(BidQty, AskQty)
    # Total profit = Profit per unit * Tradable quantity
    #
    # Market Microstructure: We can only trade the minimum of available bid
    # and ask quantities. This represents the maximum executable volume at
    # the arbitrage prices.
    #
    # Note on NaN handling: Pandas operations naturally propagate NaN:
    # - If bid_qty or ask_qty is NaN, Min() returns NaN
    # - If profit_per_unit or tradable_qty is NaN, multiplication returns NaN
    # - These NaN profits are filtered out in Step 8 (edge case handling 7)
    
    profit_per_unit = global_max_bid - global_min_ask
    tradable_qty = pd.concat([bid_qty, ask_qty], axis=1).min(axis=1)
    total_profit = profit_per_unit * tradable_qty
    
    # ========================================================================
    # STEP 7: Implement Rising Edge Detection
    # ========================================================================
    # Market Microstructure: Rising edge detection ensures we only count
    # an opportunity when it first appears (transitions from False to True).
    # If an opportunity persists for 1 second (1000 snapshots), we only
    # trade it once. Once it vanishes, we can trade it again when it reappears.
    #
    # ========================================================================
    # EDGE CASE HANDLING 6: First Timestamp in Rising Edge Detection
    # ========================================================================
    # Market Microstructure: At the very first timestamp, there is no "previous"
    # state to compare against. The .shift(1) operation returns NaN for the
    # first row, which would break boolean operations.
    #
    # Why this happens:
    # - .shift(1) shifts data down by one row, leaving first row as NaN
    # - Boolean operations with NaN return False, but we want explicit control
    #
    # How we handle it:
    # - Use .fillna(False) to treat "before first timestamp" as "no opportunity"
    # - This ensures the first opportunity (if it exists) is correctly
    #   identified as a rising edge (transition from "no opportunity" to "opportunity")
    #
    # Example: If first timestamp has an opportunity, it's a rising edge because
    # we treat the "before first" state as False (no opportunity).
    
    # Detect rising edges: opportunity exists now but didn't exist before
    # CRITICAL: fillna(False) handles the first timestamp edge case
    opportunity_prev = arbitrage_opportunity.shift(1).fillna(False)
    rising_edge = arbitrage_opportunity & ~opportunity_prev
    
    # ========================================================================
    # STEP 8: Create Signal DataFrame (only rising edges)
    # ========================================================================
    # Filter to only rising edges and create structured output
    
    signals_df = pd.DataFrame({
        'timestamp': consolidated_tape.index,
        'global_max_bid': global_max_bid,
        'global_min_ask': global_min_ask,
        'max_bid_venue': max_bid_venue,
        'min_ask_venue': min_ask_venue,
        'bid_qty': bid_qty,
        'ask_qty': ask_qty,
        'tradable_qty': tradable_qty,
        'profit_per_unit': profit_per_unit,
        'total_profit': total_profit,
        'arbitrage_opportunity': arbitrage_opportunity,
        'rising_edge': rising_edge
    })
    
    # Filter to only rising edges
    signals_df = signals_df[signals_df['rising_edge'] == True].copy()
    
    # ========================================================================
    # EDGE CASE HANDLING 7: Filter Out NaN Profits
    # ========================================================================
    # Market Microstructure: If quantities are missing (NaN), the profit
    # calculation results in NaN. We cannot trade an opportunity if we don't
    # know how much volume is available. These NaN profits appear when:
    # - Quantity columns are missing for the max bid or min ask venue
    # - Quantities themselves are NaN in the data
    # - Venue names are NaN (from edge case handling 4)
    #
    # Why we must filter them:
    # - NaN profits are not actionable: We need to know tradable volume
    # - They would skew statistics: Sum of profits would be NaN
    # - They represent incomplete data: We can't execute without quantities
    #
    # How we handle it:
    # - Filter out rows where total_profit is NaN
    # - This ensures all signals have valid, executable opportunities
    
    # CRITICAL: Filter out NaN profits (incomplete data - can't execute without quantities)
    signals_df = signals_df[signals_df['total_profit'].notna()].copy()
    
    # Set timestamp as index
    signals_df.set_index('timestamp', inplace=True)
    
    # Drop the boolean columns (no longer needed)
    signals_df = signals_df.drop(columns=['arbitrage_opportunity', 'rising_edge'])
    
    return signals_df


# ============================================================================
# EXECUTE: Generate Arbitrage Signals for All ISINs
# ============================================================================

print("\n" + "=" * 70)
print("STEP 3: GENERATING ARBITRAGE SIGNALS FOR ALL ISINs")
print("=" * 70)

# Store signals for all ISINs
arbitrage_signals = {}

# Process each ISIN
for idx, isin in enumerate(consolidated_tapes.keys(), 1):
    print(f"\n[{idx}/{len(consolidated_tapes)}] Generating signals for ISIN: {isin}")
    
    consolidated_tape = consolidated_tapes[isin]
    
    # Generate signals
    signals = generate_arbitrage_signals_for_isin(isin, consolidated_tape, all_data)
    
    if not signals.empty:
        arbitrage_signals[isin] = signals
        
        num_opportunities = len(signals)
        total_profit = signals['total_profit'].sum()
        max_profit = signals['total_profit'].max()
        
        print(f"  ✓ Found {num_opportunities:,} arbitrage opportunities (rising edges)")
        print(f"  ✓ Total theoretical profit: {total_profit:,.2f} €")
        print(f"  ✓ Maximum profit per opportunity: {max_profit:.4f} €")
        
        # Show first few opportunities as example
        if idx <= 3:  # Only show for first 3 ISINs
            print(f"\n  Sample opportunities (first 5):")
            print(signals.head().to_string())
    else:
        print(f"  ⚠️  No arbitrage opportunities found")

print("\n" + "=" * 70)
print(f"STEP 3 COMPLETE: Generated signals for {len(arbitrage_signals)} ISINs")
print("=" * 70)

# Summary statistics
if arbitrage_signals:
    total_opportunities = sum(len(signals) for signals in arbitrage_signals.values())
    total_theoretical_profit = sum(signals['total_profit'].sum() for signals in arbitrage_signals.values())
    
    print(f"\n📊 OVERALL SUMMARY:")
    print(f"   • Total ISINs with opportunities: {len(arbitrage_signals)}")
    print(f"   • Total arbitrage opportunities: {total_opportunities:,}")
    print(f"   • Total theoretical profit (0 latency): {total_theoretical_profit:,.2f} €")
else:
    print("\n⚠️  No arbitrage opportunities found across all ISINs")


### Step 4: The "Time Machine" (Latency Simulation)

- In reality, if you see a price at time $T$, you cannot trade until $T + \Delta$.
- **Task:** Simulate execution latencies of [0, 100, 500, 1000, 2000, 3000, 4000, 5000, 10000, 15000, 20000, 30000, 50000, 100000] microseconds
- *Method:* If a signal is detected at T, look up what the profit *actually is* at T + Latency in your dataframe.

In [ ]:
# ============================================================================
# STEP 4: The "Time Machine" (Latency Simulation)
# ============================================================================
# Market Microstructure Context:
# In real trading, there's a delay between detecting an opportunity and executing
# the trade. This latency can be due to network delays, order processing time,
# or exchange matching delays. By the time we execute, the opportunity may have
# disappeared or changed. This simulation helps us understand how sensitive
# our arbitrage strategy is to latency.
#
# CORRECTED IMPLEMENTATION:
# - Uses integer epoch values instead of pandas timestamps for precision
# - Uses searchsorted to find first snapshot >= execution time (not backward lookup)
# - Calculates MICs at execution time, not signal time
# - Executes trades even when spread <= 0 (realistic execution)

print("=" * 70)
print("STEP 4: LATENCY SIMULATION")
print("=" * 70)

# Define latency levels in microseconds
# These represent realistic execution delays in HFT systems
latency_levels = [0, 100, 500, 1000, 2000, 3000, 4000, 5000, 10000, 15000, 20000, 30000, 50000, 100000]

# Dictionary to store realized profits for each latency level
# Structure: {latency_us: total_realized_profit}
latency_results = {}

# ============================================================================
# OPTIMIZATION: Pre-process all latency-independent data
# ============================================================================
# All expensive operations that don't depend on latency are done ONCE here,
# before the latency loop. This includes:
# - Extending tapes with quantities
# - Calculating global_max_bid, global_min_ask
# - Calculating mic_max_bid, mic_min_ask for all rows
# - Converting timestamps to epoch
# - Extracting signal timestamps
# ============================================================================

print("Pre-processing ISINs (this is done once, before latency simulation)...")
print(f"Processing {len(arbitrage_signals)} ISINs...")

preprocessed_isins = {}
processed_count = 0

for isin, signals_df in arbitrage_signals.items():
    processed_count += 1
    if processed_count % 10 == 0:
        print(f"  Processed {processed_count}/{len(arbitrage_signals)} ISINs...")
    if signals_df.empty:
        continue
    
    # Get the consolidated tape for this ISIN
    if isin not in consolidated_tapes:
        continue
    
    consolidated_tape = consolidated_tapes[isin].copy()
    if consolidated_tape.empty:
        continue
    
    # Extend consolidated tape with quantities if needed (ONCE per ISIN)
    bid_qty_cols = [col for col in consolidated_tape.columns if '_bid_qty' in col]
    ask_qty_cols = [col for col in consolidated_tape.columns if '_ask_qty' in col]
    
    if not bid_qty_cols or not ask_qty_cols:
        consolidated_tape = extend_consolidated_tape_with_quantities(
            isin, consolidated_tape, all_data
        )
        bid_qty_cols = [col for col in consolidated_tape.columns if '_bid_qty' in col]
        ask_qty_cols = [col for col in consolidated_tape.columns if '_ask_qty' in col]
    
    # Get bid and ask price columns (ONCE per ISIN)
    bid_cols = [col for col in consolidated_tape.columns if '_bid' in col and '_qty' not in col]
    ask_cols = [col for col in consolidated_tape.columns if '_ask' in col and '_qty' not in col]
    
    if not bid_cols or not ask_cols:
        continue
    
    # Calculate global_max_bid and global_min_ask (ONCE per ISIN)
    if 'global_max_bid' not in consolidated_tape.columns:
        consolidated_tape['global_max_bid'] = consolidated_tape[bid_cols].max(axis=1)
    if 'global_min_ask' not in consolidated_tape.columns:
        consolidated_tape['global_min_ask'] = consolidated_tape[ask_cols].min(axis=1)
    
    # Get new_opportunity signals (ONCE per ISIN)
    if 'new_opportunity' in signals_df.columns:
        signal_indices = signals_df[signals_df['new_opportunity']].index
    else:
        # Calculate new_opportunity if not present
        arbitrage_opportunity = consolidated_tape['global_max_bid'] > consolidated_tape['global_min_ask']
        arbitrage_prev = arbitrage_opportunity.shift(1, fill_value=False)
        new_opportunity = arbitrage_opportunity & ~arbitrage_prev
        signal_indices = consolidated_tape[new_opportunity].index
    
    if len(signal_indices) == 0:
        continue
    
    # Convert timestamp index to epoch microseconds (ONCE per ISIN)
    if consolidated_tape.index.dtype == 'datetime64[ns]' or (len(consolidated_tape) > 0 and isinstance(consolidated_tape.index[0], pd.Timestamp)):
        tape_epoch = consolidated_tape.index.astype('int64') // 1000  # nanoseconds to microseconds
    else:
        tape_epoch = pd.Series(consolidated_tape.index).astype('int64')
    
    tape_epoch_values = tape_epoch.values  # numpy array for fast searchsorted
    
    # Convert signal timestamps to epoch microseconds (ONCE per ISIN)
    signal_epochs = []
    for signal_timestamp in signal_indices:
        if isinstance(signal_timestamp, pd.Timestamp):
            t_signal = int(signal_timestamp.value // 1000)
        else:
            t_signal = int(signal_timestamp)
        signal_epochs.append(t_signal)
    signal_epochs = np.array(signal_epochs)  # numpy array for vectorized operations
    
    # Calculate MICs for best bid/ask at each timestamp (ONCE per ISIN, for ALL rows)
    mic_max_bid = pd.Series(index=consolidated_tape.index, dtype="object")
    mic_min_ask = pd.Series(index=consolidated_tape.index, dtype="object")
    
    valid_bid_rows = consolidated_tape[bid_cols].notna().any(axis=1)
    mic_max_bid.loc[valid_bid_rows] = (
        consolidated_tape.loc[valid_bid_rows, bid_cols]
        .idxmax(axis=1)
        .str.replace('_bid', '', regex=False)
    )
    
    valid_ask_rows = consolidated_tape[ask_cols].notna().any(axis=1)
    mic_min_ask.loc[valid_ask_rows] = (
        consolidated_tape.loc[valid_ask_rows, ask_cols]
        .idxmin(axis=1)
        .str.replace('_ask', '', regex=False)
    )
    
    # Store preprocessed data for this ISIN
    preprocessed_isins[isin] = {
        'tape': consolidated_tape,
        'epoch_values': tape_epoch_values,
        'signal_epochs': signal_epochs,
        'signal_indices': signal_indices,
        'mic_max_bid': mic_max_bid,
        'mic_min_ask': mic_min_ask,
        'bid_qty_cols': bid_qty_cols,
        'ask_qty_cols': ask_qty_cols,
        'n': len(consolidated_tape)
    }

print(f"Pre-processing complete: {len(preprocessed_isins)} ISINs ready for latency simulation\n")

# ============================================================================
# OPTIMIZED LATENCY LOOP
# ============================================================================
# Now we only loop through latencies and signals, using preprocessed data
# ============================================================================

# Process each latency level
for latency_us in latency_levels:
    print(f"\n[Processing Latency: {latency_us} microseconds]")
    
    total_realized_profit = 0.0
    total_opportunities = 0
    
    # Process each ISIN using preprocessed data
    for isin, preprocessed in preprocessed_isins.items():
        consolidated_tape = preprocessed['tape']
        tape_epoch_values = preprocessed['epoch_values']
        signal_epochs = preprocessed['signal_epochs']
        signal_indices = preprocessed['signal_indices']
        mic_max_bid = preprocessed['mic_max_bid']
        mic_min_ask = preprocessed['mic_min_ask']
        n = preprocessed['n']
        
        # Process each signal (new opportunity)
        isin_profit = 0.0
        isin_opportunities = 0
        
        for i, signal_epoch in enumerate(signal_epochs):
            # Calculate target execution time (simple addition)
            t_exec_target = signal_epoch + latency_us
            
            # Find first snapshot with epoch >= t_exec_target using searchsorted
            pos = np.searchsorted(tape_epoch_values, t_exec_target, side='left')
            
            if pos >= n:
                continue  # No data at or after t_exec_target
            
            # Get execution row (using iloc for faster access)
            row_exec = consolidated_tape.iloc[pos]
            
            # Prices at execution time
            max_bid = row_exec['global_max_bid']
            min_ask = row_exec['global_min_ask']
            spread = max_bid - min_ask
            
            # Get MICs at execution time (not signal time!)
            buy_mic = mic_min_ask.iloc[pos]
            sell_mic = mic_max_bid.iloc[pos]
            
            # Get quantities at execution time
            if pd.notna(buy_mic) and pd.notna(sell_mic):
                bid_qty_col = f'{sell_mic}_bid_qty'
                ask_qty_col = f'{buy_mic}_ask_qty'
                
                bid_qty = row_exec.get(bid_qty_col, np.nan) if bid_qty_col in consolidated_tape.columns else np.nan
                ask_qty = row_exec.get(ask_qty_col, np.nan) if ask_qty_col in consolidated_tape.columns else np.nan
                
                # How much we can trade
                trade_qty = min(bid_qty, ask_qty) if pd.notna(bid_qty) and pd.notna(ask_qty) else 0.0
                
                if np.isnan(trade_qty) or trade_qty <= 0:
                    profit = 0.0
                else:
                    # ALWAYS execute, even if spread <= 0 (realistic execution)
                    profit = spread * trade_qty
            else:
                profit = 0.0
            
            isin_profit += profit
            isin_opportunities += 1
        
        total_realized_profit += isin_profit
        total_opportunities += isin_opportunities
    
    # Store results for this latency level
    latency_results[latency_us] = {
        'total_realized_profit': total_realized_profit,
        'total_opportunities': total_opportunities
    }
    
    print(f"  Total Realized Profit: {total_realized_profit:,.2f} €")
    print(f"  Total Opportunities Processed: {total_opportunities:,}")

print("\n" + "=" * 70)
print("STEP 4 COMPLETE: Latency simulation finished")
print("=" * 70)


## 4. Deliverables & Evaluation

Submit a Jupyter Notebook containing your code and the following analysis:

1. **The "Money Table":** A summary table showing the Total Realized Profit for all processed ISINs at each latency level.
2. **The Decay Chart:** A line chart visualizing how Total Profit (Y-axis) decays as Latency (X-axis) increases.
3. **Top Opportunities:** A list of the Top 5 most profitable ISINs (at 0 latency). **Sanity check these results**—do they look real?

### 1. The "Money Table"

In [ ]:
# ============================================================================
# DELIVERABLE 1: The "Money Table"
# ============================================================================
# Market Microstructure Context:
# This table summarizes the total realized profit at each latency level.
# It shows how quickly arbitrage opportunities disappear as execution latency
# increases. This is critical for HFT firms to understand the minimum
# latency requirements for profitability.

# Create the Money Table DataFrame
money_table_data = []
for latency_us in latency_levels:
    result = latency_results[latency_us]
    money_table_data.append({
        'Latency (µs)': latency_us,
        'Total Realized Profit (€)': result['total_realized_profit'],
        'Number of Opportunities': result['total_opportunities']
    })

money_table = pd.DataFrame(money_table_data)

# Format for better readability
money_table['Total Realized Profit (€)'] = money_table['Total Realized Profit (€)'].apply(lambda x: f"{x:,.2f}")

# Display the table
print("=" * 70)
print("THE MONEY TABLE: Total Realized Profit by Latency Level")
print("=" * 70)
print(money_table.to_string(index=False))
print("\n")

# Also store the numeric version for the chart
money_table_numeric = pd.DataFrame(money_table_data)


### 2. The Decay Chart

In [ ]:
# ============================================================================
# DELIVERABLE 2: The Decay Chart
# ============================================================================
# Market Microstructure Context:
# This visualization shows the "latency decay curve" - how quickly arbitrage
# profits disappear as execution latency increases. This is a critical metric
# for HFT firms: it shows the maximum acceptable latency before the strategy
# becomes unprofitable. The steeper the decay, the more latency-sensitive
# the arbitrage opportunity.

# Create the decay chart
fig, ax = plt.subplots(figsize=(12, 6))

# Extract data for plotting
latencies = money_table_numeric['Latency (µs)']
profits = money_table_numeric['Total Realized Profit (€)']

# Plot the decay curve
ax.plot(latencies, profits, marker='o', linewidth=2, markersize=8, color='#2E86AB')

# Formatting
ax.set_xlabel('Latency (microseconds)', fontsize=12, fontweight='bold')
ax.set_ylabel('Total Realized Profit (€)', fontsize=12, fontweight='bold')
ax.set_title('Latency Decay Curve: Profit vs Execution Latency', fontsize=14, fontweight='bold', pad=20)

# Add grid for better readability
ax.grid(True, linestyle='--', alpha=0.7, linewidth=0.5)

# Format x-axis to show latency in a readable format
ax.set_xscale('linear')
# Add minor ticks for better granularity
ax.xaxis.set_minor_locator(plt.MultipleLocator(5000))

# Format y-axis to show currency with commas
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'{x:,.0f}'))

# Add annotations for key points (0 latency and where profit drops significantly)
if len(profits) > 0:
    # Annotate 0 latency point
    ax.annotate(f'€{profits.iloc[0]:,.2f}', 
                xy=(latencies.iloc[0], profits.iloc[0]),
                xytext=(10, 10), textcoords='offset points',
                bbox=dict(boxstyle='round,pad=0.5', facecolor='yellow', alpha=0.7),
                arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0'),
                fontsize=10, fontweight='bold')
    
    # Find where profit drops to 50% of initial (if applicable)
    if profits.iloc[0] > 0:
        half_profit = profits.iloc[0] / 2
        for i, profit in enumerate(profits):
            if profit <= half_profit:
                ax.annotate(f'50% Decay\n€{profit:,.2f}',
                           xy=(latencies.iloc[i], profit),
                           xytext=(10, -30), textcoords='offset points',
                           bbox=dict(boxstyle='round,pad=0.5', facecolor='orange', alpha=0.7),
                           arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0'),
                           fontsize=9)
                break

plt.tight_layout()
plt.show()

print("=" * 70)
print("DECAY CHART GENERATED: Visualizing profit decay with latency")
print("=" * 70)


### 3. Top Opportunities

In [ ]:
# ============================================================================
# DELIVERABLE 3: Top 5 Opportunities (at 0 Latency)
# ============================================================================
# Market Microstructure Context:
# This identifies the most profitable ISINs for arbitrage at 0 latency
# (theoretical maximum). These are the instruments where price discrepancies
# between venues are largest and most frequent. Sanity checking these results
# is critical: we need to verify that the profits are realistic and not due
# to data errors or off-book trades.

# Calculate total profit per ISIN at 0 latency
isin_profits = []
for isin, signals_df in arbitrage_signals.items():
    if not signals_df.empty:
        total_profit = signals_df['total_profit'].sum()
        num_opportunities = len(signals_df)
        avg_profit_per_opportunity = total_profit / num_opportunities if num_opportunities > 0 else 0
        max_profit = signals_df['total_profit'].max()
        
        isin_profits.append({
            'ISIN': isin,
            'Total Profit (€)': total_profit,
            'Number of Opportunities': num_opportunities,
            'Avg Profit per Opportunity (€)': avg_profit_per_opportunity,
            'Max Profit per Opportunity (€)': max_profit
        })

# Create DataFrame and sort by total profit
top_opportunities_df = pd.DataFrame(isin_profits)
top_opportunities_df = top_opportunities_df.sort_values('Total Profit (€)', ascending=False)

# Get top 5
top_5_opportunities = top_opportunities_df.head(5).copy()

# Format for display
top_5_display = top_5_opportunities.copy()
top_5_display['Total Profit (€)'] = top_5_display['Total Profit (€)'].apply(lambda x: f"{x:,.2f}")
top_5_display['Avg Profit per Opportunity (€)'] = top_5_display['Avg Profit per Opportunity (€)'].apply(lambda x: f"{x:.4f}")
top_5_display['Max Profit per Opportunity (€)'] = top_5_display['Max Profit per Opportunity (€)'].apply(lambda x: f"{x:.4f}")

# Display results
print("=" * 70)
print("TOP 5 MOST PROFITABLE ISINs (at 0 Latency)")
print("=" * 70)
print(top_5_display.to_string(index=False))
print("\n")

# Sanity Check Analysis
print("=" * 70)
print("SANITY CHECK ANALYSIS")
print("=" * 70)
print("\n1. Profit Magnitude Check:")
print("   - Are the profit values realistic for arbitrage?")
print("   - Typical arbitrage profits are small (cents per share)")
print("   - Large profits might indicate data quality issues\n")

print("2. Opportunity Frequency Check:")
for idx, row in top_5_opportunities.iterrows():
    isin = row['ISIN']
    num_opps = int(row['Number of Opportunities'])
    avg_profit = row['Avg Profit per Opportunity (€)']
    print(f"   - {isin}: {num_opps:,} opportunities, avg profit: €{avg_profit:.4f}")

print("\n3. Data Quality Check:")
print("   - Verify that these ISINs have sufficient liquidity")
print("   - Check that prices are from lit order books (not off-book)")
print("   - Ensure timestamps are correctly synchronized across venues\n")

print("4. Market Microstructure Validation:")
print("   - High-frequency arbitrage opportunities are typically:")
print("     * Very small profit per trade (€0.01 - €0.10)")
print("     * Very frequent (hundreds or thousands per day)")
print("     * Highly sensitive to latency (disappear in microseconds)\n")

print("=" * 70)


## Grading Rubric (Max 10 Points)

- **5-6 Points (Baseline):** The code runs, correctly calculates the consolidated tape, identifies Bid > Ask opportunities, and estimates theoretical (0 latency) profit.

- **7-8 Points (Robust):** The simulation accurately models latency (using strict time-lookups) and strictly adheres to the vendor's data quality specs.

- **9-10 Points (Expert):** You demonstrate deep understanding of market microstructure. You handle **Market Status** correctly to avoid fake signals, identify anomalies in the instrument list, and handle edge cases around Market Open/Close.